# 0️⃣ InteractivePipelineLoadFromPickle (Independent Load-only Visualization Notebook) - Imports

In [33]:
%config IPCompleter.use_jedi = False
# %xmode Verbose
# %xmode context
%pdb off
%load_ext autoreload
%autoreload 3
# # !pip install viztracer
# %load_ext viztracer
# from viztracer import VizTracer

# %load_ext memory_profiler

import sys
from pathlib import Path

# required to enable non-blocking interaction:
%gui qt5

import importlib
from copy import deepcopy
from numba import jit
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
# pd.options.mode.dtype_backend = 'pyarrow' # use new pyarrow backend instead of numpy
from attrs import define, field, fields, Factory, make_class
import tables as tb
from datetime import datetime, timedelta

# Pho's Formatting Preferences
import builtins

import IPython
from IPython.core.formatters import PlainTextFormatter
from IPython import get_ipython

from pyphocorehelpers.preferences_helpers import set_pho_preferences, set_pho_preferences_concise, set_pho_preferences_verbose
set_pho_preferences_concise()
# # Jupyter-lab enable printing for any line on its own (instead of just the last one in the cell)
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"

# BEGIN PPRINT CUSTOMIZATION ___________________________________________________________________________________________ #

## IPython pprint
from pyphocorehelpers.pprint import wide_pprint, wide_pprint_ipython, wide_pprint_jupyter, MAX_LINE_LENGTH
# Override default pprint
builtins.pprint = wide_pprint

ip = get_ipython()

from pyphocorehelpers.ipython_helpers import CustomFormatterMagics

# Register the magic
get_ipython().register_magics(CustomFormatterMagics)

text_formatter: PlainTextFormatter = ip.display_formatter.formatters['text/plain']
text_formatter.max_width = MAX_LINE_LENGTH
text_formatter.for_type(object, wide_pprint_jupyter)


# END PPRINT CUSTOMIZATION ___________________________________________________________________________________________ #

from pyphocorehelpers.print_helpers import get_now_time_str, get_now_day_str
from pyphocorehelpers.indexing_helpers import get_dict_subset

## Pho's Custom Libraries:
from pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path, file_uri_from_path
from pyphocorehelpers.Filesystem.open_in_system_file_manager import reveal_in_system_file_manager
import pyphocorehelpers.programming_helpers as programming_helpers

# NeuroPy (Diba Lab Python Repo) Loading
# from neuropy import core
from typing import Dict, List, Tuple, Optional, Callable, Union, Any
from typing_extensions import TypeAlias
from nptyping import NDArray
import neuropy.utils.type_aliases as types

from neuropy.analyses.placefields import PlacefieldComputationParameters
from neuropy.core.epoch import NamedTimerange, Epoch
from neuropy.core.ratemap import Ratemap
from neuropy.core.session.Formats.BaseDataSessionFormats import DataSessionFormatRegistryHolder, DataSessionFormatBaseRegisteredClass
# from neuropy.core.session.Formats.Specific.KDibaOldDataSessionFormat import KDibaOldDataSessionFormatRegisteredClass
# from neuropy.core.session.Formats.Specific.BapunDataSessionFormat import BapunDataSessionFormatRegisteredClass
# from neuropy.core.session.Formats.Specific.RachelDataSessionFormat import RachelDataSessionFormat
from neuropy.core.session.Formats.Specific.BapunDataSessionFormat import BapunDataSessionFormatRegisteredClass

from neuropy.utils.matplotlib_helpers import matplotlib_file_only, matplotlib_configuration, matplotlib_configuration_update
from neuropy.core.neuron_identities import NeuronIdentityTable, neuronTypesList, neuronTypesEnum
from neuropy.utils.mixins.AttrsClassHelpers import AttrsBasedClassHelperMixin, serialized_field, serialized_attribute_field, non_serialized_field, custom_define
from neuropy.utils.mixins.HDF5_representable import HDF_DeserializationMixin, post_deserialize, HDF_SerializationMixin, HDFMixin, HDF_Converter

## For computation parameters:
from neuropy.analyses.placefields import PlacefieldComputationParameters
from neuropy.utils.dynamic_container import DynamicContainer
from neuropy.utils.result_context import IdentifyingContext
from neuropy.core.session.Formats.BaseDataSessionFormats import find_local_session_paths
from neuropy.core.user_annotations import UserAnnotationsManager

from pyphocorehelpers.print_helpers import print_object_memory_usage, print_dataframe_memory_usage, print_value_overview_only, DocumentationFilePrinter, print_keys_if_possible, generate_html_string, document_active_variables
from pyphocorehelpers.programming_helpers import metadata_attributes
from pyphocorehelpers.function_helpers import function_attributes
## Pho Programming Helpers:
from pyphocorehelpers.print_helpers import DocumentationFilePrinter, TypePrintMode, print_keys_if_possible, debug_dump_object_member_shapes, print_value_overview_only, document_active_variables
from pyphocorehelpers.programming_helpers import IPythonHelpers, PythonDictionaryDefinitionFormat, MemoryManagement, inspect_callable_arguments, get_arguments_as_optional_dict, GeneratedClassDefinitionType, CodeConversion
from pyphocorehelpers.notebook_helpers import NotebookCellExecutionLogger
from pyphocorehelpers.gui.Qt.TopLevelWindowHelper import TopLevelWindowHelper, print_widget_hierarchy
from pyphocorehelpers.indexing_helpers import reorder_columns, reorder_columns_relative, dict_to_full_array
from pyphocorehelpers.DataStructure.RenderPlots.MatplotLibRenderPlots import MatplotlibRenderPlots

doc_output_parent_folder: Path = Path('EXTERNAL/DEVELOPER_NOTES/DataStructureDocumentation').resolve() # ../.
print(f"doc_output_parent_folder: {doc_output_parent_folder}")
assert doc_output_parent_folder.exists()

_notebook_path:Path = Path(IPythonHelpers.try_find_notebook_filepath(IPython.extract_module_locals())).resolve() # Finds the path of THIS notebook
# _notebook_execution_logger: NotebookCellExecutionLogger = NotebookCellExecutionLogger(notebook_path=_notebook_path, enable_logging_to_file=False) # Builds a logger that records info about this notebook

# pyPhoPlaceCellAnalysis:
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import NeuropyPipeline # get_neuron_identities
from pyphoplacecellanalysis.General.Mixins.ExportHelpers import export_pyqtgraph_plot
from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_load_session, batch_extended_computations, batch_evaluate_required_computations
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import PipelineSavingScheme # used in perform_pipeline_save
from pyphoplacecellanalysis.GUI.IPyWidgets.pipeline_ipywidgets import PipelineJupyterHelpers, CustomProcessingPhases
from pyphocorehelpers.assertion_helpers import Assert

import pyphoplacecellanalysis.External.pyqtgraph as pg

from pyphocorehelpers.exception_helpers import ExceptionPrintingContext, CapturedException
from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_perform_all_plots
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import JonathanFiringRateAnalysisResult
from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import _find_any_context_neurons
from pyphoplacecellanalysis.General.Batch.runBatch import BatchSessionCompletionHandler # for `post_compute_validate(...)`
from pyphoplacecellanalysis.Analysis.Decoder.reconstruction import BasePositionDecoder
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionsResults
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.SpikeAnalysis import SpikeRateTrends # for `_perform_long_short_instantaneous_spike_rate_groups_analysis`
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import SingleBarResult, InstantaneousSpikeRateGroupsComputation, TruncationCheckingResults # for `BatchSessionCompletionHandler`, `AcrossSessionsAggregator`
from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import SplitPartitionMembership
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.DirectionalPlacefieldGlobalComputationFunctions import DirectionalPlacefieldGlobalComputationFunctions, DirectionalLapsResult, TrackTemplates, DecoderDecodedEpochsResult
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.RankOrderComputations import RankOrderGlobalComputationFunctions,  RankOrderComputationsContainer, RankOrderResult, RankOrderAnalyses
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.DirectionalPlacefieldGlobalComputationFunctions import TrackTemplates
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.ComputationFunctionRegistryHolder import ComputationFunctionRegistryHolder, computation_precidence_specifying_function, global_function
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.SequenceBasedComputations import WCorrShuffle, SequenceBasedComputationsContainer
from neuropy.utils.mixins.binning_helpers import transition_matrix
from pyphoplacecellanalysis.Analysis.Decoder.transition_matrix import TransitionMatrixComputations
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.DirectionalPlacefieldGlobalComputationFunctions import TrackTemplates, get_proper_global_spikes_df
from pyphocorehelpers.Filesystem.path_helpers import set_posix_windows
from pyphoplacecellanalysis.Analysis.Decoder.reconstruction import BasePositionDecoder, DecodedFilterEpochsResult, SingleEpochDecodedResult

from pyphocorehelpers.assertion_helpers import Assert

# Plotting
# import pylustrator # customization of figures
import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
_bak_rcParams = mpl.rcParams.copy()

matplotlib.use('Qt5Agg')
# %matplotlib inline
# %matplotlib auto

# _restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
_restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')

import seaborn as sns

# import pylustrator # call `pylustrator.start()` before creating your first figure in code.
from pyphoplacecellanalysis.Pho2D.matplotlib.visualize_heatmap import visualize_heatmap, visualize_heatmap_pyqtgraph # used in `plot_kourosh_activity_style_figure`
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.SpikeRasters import plot_multiple_raster_plot, plot_raster_plot
from pyphoplacecellanalysis.General.Mixins.DataSeriesColorHelpers import UnitColoringMode, DataSeriesColorHelpers
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.SpikeRasters import _build_default_tick, build_scatter_plot_kwargs
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.Mixins.Render2DScrollWindowPlot import Render2DScrollWindowPlotMixin, ScatterItemData
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.SpikeAnalysis import SpikeRateTrends
from pyphoplacecellanalysis.General.Mixins.SpikesRenderingBaseMixin import SpikeEmphasisState
from pyphoplacecellanalysis.General.Model.SpecificComputationParameterTypes import ComputationKWargParameters
from pyphoplacecellanalysis.SpecificResults.PhoDiba2023Paper import PAPER_FIGURE_figure_1_add_replay_epoch_rasters, PAPER_FIGURE_figure_1_full, PAPER_FIGURE_figure_3, main_complete_figure_generations
# from pyphoplacecellanalysis.SpecificResults.fourthYearPresentation import *

# Jupyter Widget Interactive
import ipywidgets as widgets
from IPython.display import display, HTML
from pyphocorehelpers.Filesystem.open_in_system_file_manager import reveal_in_system_file_manager
from pyphoplacecellanalysis.GUI.IPyWidgets.pipeline_ipywidgets import interactive_pipeline_widget, interactive_pipeline_files
from pyphocorehelpers.gui.Jupyter.simple_widgets import fullwidth_path_widget, render_colors

from datetime import datetime, date, timedelta
from pyphocorehelpers.print_helpers import get_now_day_str, get_now_rounded_time_str

from neuropy.core.session.Formats.BaseDataSessionFormats import HardcodedProcessingParameters

known_data_session_type_properties_dict = DataSessionFormatRegistryHolder.get_registry_known_data_session_type_dict()
active_data_session_types_registered_classes_dict = DataSessionFormatRegistryHolder.get_registry_data_session_type_class_name_dict()

DAY_DATE_STR: str = date.today().strftime("%Y-%m-%d")
DAY_DATE_TO_USE = f'{DAY_DATE_STR}' # used for filenames throught the notebook
print(f'DAY_DATE_STR: {DAY_DATE_STR}, DAY_DATE_TO_USE: {DAY_DATE_TO_USE}')

NOW_DATETIME: str = get_now_rounded_time_str()
NOW_DATETIME_TO_USE = f'{NOW_DATETIME}' # used for filenames throught the notebook
print(f'NOW_DATETIME: {NOW_DATETIME}, NOW_DATETIME_TO_USE: {NOW_DATETIME_TO_USE}')

def get_global_variable(var_name):
    """ used by `PipelineJupyterHelpers._build_pipeline_custom_processing_mode_selector_widget(...)` to update the notebook's variables """
    return globals()[var_name]
    
def update_global_variable(var_name, value):
    """ used by `PipelineJupyterHelpers._build_pipeline_custom_processing_mode_selector_widget(...)` to update the notebook's variables """
    globals()[var_name] = value

from pyphocorehelpers.gui.Jupyter.simple_widgets import build_global_data_root_parent_path_selection_widget
all_paths = [Path(r'/home/halechr/FastData'), Path('/Volumes/SwapSSD/Data'), Path('/Users/pho/data'), Path(r'/media/halechr/MAX/Data'), Path(r'H:\Data'), Path(r'W:\Data'), Path(r'/home/halechr/cloud/turbo/Data'), Path(r'/Volumes/MoverNew/data'), Path(r'/home/halechr/turbo/Data'), Path(r'/Users/pho/cloud/turbo/Data')] # Path('/Volumes/FedoraSSD/FastData'), 
global_data_root_parent_path = None
def on_user_update_path_selection(new_path: Path):
    global global_data_root_parent_path
    new_global_data_root_parent_path = new_path.resolve()
    global_data_root_parent_path = new_global_data_root_parent_path
    print(f'global_data_root_parent_path changed to {global_data_root_parent_path}')
    assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"
            
global_data_root_parent_path_widget = build_global_data_root_parent_path_selection_widget(all_paths, on_user_update_path_selection)
global_data_root_parent_path_widget

Automatic pdb calling has been turned OFF
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
doc_output_parent_folder: C:\Users\pho\repos\Spike3DWorkEnv\Spike3D\EXTERNAL\DEVELOPER_NOTES\DataStructureDocumentation
DAY_DATE_STR: 2025-11-03, DAY_DATE_TO_USE: 2025-11-03
NOW_DATETIME: 2025-11-03_1225PM, NOW_DATETIME_TO_USE: 2025-11-03_1225PM
global_data_root_parent_path changed to W:\Data


ToggleButtons(description='Data Root:', layout=Layout(width='auto'), options=(WindowsPath('W:/Data'),), style=ToggleButtonsStyle(button_width='max-content'), tooltip='global_data_root_parent_path', value=WindowsPath('W:/Data'))

# 0️⃣ Load Pipeline

In [34]:
# ==================================================================================================================== #
# Load Data                                                                                                            #
# ==================================================================================================================== #

# ==================================================================================================================== #
# BAPUN data format                                                                                                    #
# ==================================================================================================================== #
active_data_mode_name = 'bapun'
local_session_root_parent_context = IdentifyingContext(format_name=active_data_mode_name) # , animal_name='', configuration_name='one', session_name=a_sess.session_name
local_session_root_parent_path = global_data_root_parent_path.joinpath('Bapun')

# [*] - indicates bad or session with a problem
# 0, 1, 2, 3, 4, 5, 6, 7, [8], [9], 10, 11, [12], 13, 14, [15], [16], 17, 
curr_context = IdentifyingContext(format_name='bapun',animal='RatN', session_name='Day4OpenField') # ,exper_name='one' -- This is the one with the 2 behaviors ('roam', 'sprinkle') in the same open field box
# curr_context = IdentifyingContext(format_name='bapun',animal='RatS', session_name='Day5TwoNovel') # ,exper_name='one' -- this one has the 2 maze shapes: ["N", "U"]
# Create a dictionary with the parameters to override
override_parameters = {
    'preprocessing.laps.use_direction_dependent_laps': False
}

local_session_parent_path: Path = local_session_root_parent_path.joinpath(curr_context.animal) # 'gor01', 'one' - probably not needed anymore
basedir: Path = local_session_parent_path.joinpath(curr_context.session_name) #.resolve()

# basedir: Path = Path('/media/halechr/MAX/Data/Rachel/cho/cho_241117_2_merged') # DO NOT `.resolve()``
# basedir: Path = Path(r'H:\Data\Bapun\RatS\Day5TwoNovel')
print(f'basedir: {str(basedir)}')
Assert.path_exists(basedir)

epoch_name_includelist = None
active_computation_functions_name_includelist = ['pf_computation', 'pfdt_computation', 'position_decoding']

# Read if possible:
saving_mode = PipelineSavingScheme.SKIP_SAVING
force_reload = False

# # Force write:
# saving_mode = PipelineSavingScheme.TEMP_THEN_OVERWRITE
# saving_mode = PipelineSavingScheme.OVERWRITE_IN_PLACE
# force_reload = True

selector, on_value_change = PipelineJupyterHelpers._build_pipeline_custom_processing_mode_selector_widget(update_global_variable_fn=update_global_variable, debug_print=False, enable_full_view=True)
# selector.value = 'clean_run'
selector.value = 'continued_run'
# selector.value = 'final_run'
on_value_change(dict(new=selector.value)) ## do update manually so the workspace variables reflect the set values
## TODO: if loading is not possible, we need to change the `saving_mode` so that the new results are properly saved.
print(f"saving_mode: {saving_mode}, force_reload: {force_reload}")

basedir: W:\Data\Bapun\RatN\Day4OpenField


saving_mode: PipelineSavingScheme.SKIP_SAVING, force_reload: False


# Resume

In [35]:
from pyphoplacecellanalysis.GUI.IPyWidgets.pipeline_ipywidgets import PipelineJupyterHelpers, CustomProcessingPhases, PipelinePickleFileSelectorWidget

# ## INPUTS: basedir
# active_session_pickle_file_widget = PipelinePickleFileSelectorWidget(directory=basedir)

extended_computations_include_includelist_phase_dict: Dict[str, CustomProcessingPhases] = CustomProcessingPhases.get_extended_computations_include_includelist_phase_dict()

current_phase: CustomProcessingPhases = CustomProcessingPhases[selector.value]  # Assuming selector.value is an instance of CustomProcessingPhases
extended_computations_include_includelist: List[str] = [key for key, value in extended_computations_include_includelist_phase_dict.items() if value <= current_phase]
display(extended_computations_include_includelist)
force_recompute_override_computations_includelist = None
# force_recompute_override_computations_includelist = ['split_to_directional_laps', 'merged_directional_placefields', 'rank_order_shuffle_analysis', 'directional_decoders_decode_continuous'] # 

# ## INPUTS: basedir
active_session_pickle_file_widget = PipelinePickleFileSelectorWidget(directory=basedir, on_update_global_variable_callback=update_global_variable, on_get_global_variable_callback=get_global_variable)

_subfn_load, _subfn_save, _subfn_compute, _subfn_compute_new = active_session_pickle_file_widget._build_load_save_callbacks(global_data_root_parent_path=global_data_root_parent_path, active_data_mode_name=active_data_mode_name, basedir=basedir, saving_mode=saving_mode, force_reload=force_reload,
                                                             extended_computations_include_includelist=extended_computations_include_includelist, force_recompute_override_computations_includelist=force_recompute_override_computations_includelist)


# Display the widget
display(active_session_pickle_file_widget.servable())
# active_session_pickle_file_widget.local_file_browser_widget.servable()
# active_session_pickle_file_widget.global_file_browser_widget.servable()
# display(active_session_pickle_file_widget.local_file_browser_widget.servable())
# display(active_session_pickle_file_widget.global_file_browser_widget.servable())

# OUTPUTS: active_session_pickle_file_widget, widget.active_local_pkl, widget.active_global_pkl

if selector.value == 'clean_run':
    ## handle a clean run specially, this will create the pkls and not load them
    print(f'clean run!')
    default_selected_local_file_name: str = 'loadedSessPickle.pkl'
    default_selected_global_file_name: str = 'global_computation_results.pkl'
    # active_session_pickle_file_widget.is_compute_button_disabled = False # enable the compute button always during a clean run
    # active_session_pickle_file_widget.is_load_button_disabled = True
    
    new_default_local_pkl_file: Path = active_session_pickle_file_widget.directory.joinpath(default_selected_local_file_name).resolve()
    print(f'new_default_local_pkl_file: {new_default_local_pkl_file}')

    active_session_pickle_file_widget.selected_local_pkl_files = [new_default_local_pkl_file]
    active_session_pickle_file_widget.selected_global_pkl_files = []
    active_session_pickle_file_widget._update_load_save_button_disabled_state()
    print(f'active_session_pickle_file_widget.is_load_button_disabled: {active_session_pickle_file_widget.is_load_button_disabled}')
    print(f'active_session_pickle_file_widget.is_compute_button_disabled: {active_session_pickle_file_widget.is_compute_button_disabled}')
    print(f'active_local_pkl: "{active_session_pickle_file_widget.active_local_pkl}"')
    print(f'active_global_pkl: "{active_session_pickle_file_widget.active_global_pkl}"')
    active_session_pickle_file_widget.load_button.disabled = False
    active_session_pickle_file_widget.compute_button.disabled = False
else:
    # not `clean_run` mode, continuing processing which might include loading from pickles
    ## try selecting the first
    did_find_valid_selection: bool = active_session_pickle_file_widget.try_select_first_valid_files()

    ## Set default local comp pkl:
    default_selected_local_file_name: str = 'loadedSessPickle.pkl'
    if not active_session_pickle_file_widget.is_local_file_names_list_empty:
        default_local_section_indicies = [active_session_pickle_file_widget.local_file_browser_widget._data['File Name'].tolist().index(default_selected_local_file_name)]
        active_session_pickle_file_widget.local_file_browser_widget.selection = default_local_section_indicies

    ## Set default global computation pkl:
    default_selected_global_file_name: str = 'global_computation_results.pkl'
    if not active_session_pickle_file_widget.is_global_file_names_list_empty:
        default_global_section_indicies = [active_session_pickle_file_widget.global_file_browser_widget._data['File Name'].tolist().index(default_selected_global_file_name)]
        active_session_pickle_file_widget.global_file_browser_widget.selection = default_global_section_indicies



['lap_direction_determination',
 'pf_computation',
 'pfdt_computation',
 'position_decoding',
 'firing_rate_trends',
 'extended_stats',
 'long_short_decoding_analyses',
 'jonathan_firing_rate_analysis',
 'long_short_fr_indicies_analyses',
 'long_short_post_decoding',
 'long_short_inst_spike_rate_groups',
 'long_short_endcap_analysis',
 'split_to_directional_laps',
 'merged_directional_placefields',
 'directional_decoders_decode_continuous',
 'directional_decoders_evaluate_epochs',
 'directional_decoders_epoch_heuristic_scoring',
 'non_PBE_epochs_results',
 'generalized_specific_epochs_decoding']

Column
    [0] Tabulator(disabled=True, height=400, page_size=10, pagination='local', show_index=False, sorters=[{'field': 'Modification D...], value=              ...)
    [1] Tabulator(disabled=True, height=400, page_size=10, pagination='local', show_index=False, sorters=[{'field': 'Modification D...], value=              ...)
    [2] Column(margin=(10, 0))
        [0] HTML(str)
        [1] Row
            [0] IntInput(end=100, name='Min FR (Hz)', start=0, value=2, width=150)
            [1] TextInput(name='QClu Values', placeholder='Enter as list: [1, ..., value='[1, 2, 4, 6, 7, 8, 9]')
    [3] Row
        [0] Button(button_type='success', name='Save')
        [1] Button(button_type='primary', disabled=True, name='Load')
        [2] Button(button_type='warning', name='Compute')
        [3] Button(button_type='primary', name='Compute New')

In [9]:
did_find_valid_selection: bool = active_session_pickle_file_widget.try_select_first_valid_files()
did_find_valid_selection


True

In [ ]:
if did_find_valid_selection:
    _subfn_load()


In [ ]:
if did_find_valid_selection:
    curr_active_pipeline, custom_suffix, proposed_load_pkl_path = active_session_pickle_file_widget.on_load_local(global_data_root_parent_path=global_data_root_parent_path, active_data_mode_name=active_data_mode_name, basedir=basedir, saving_mode=saving_mode, force_reload=force_reload)
    print(f'on_load_local(...) complete. workspace variables updated: curr_active_pipeline, custom_suffix, proposed_load_pkl_path')
    

In [ ]:
if did_find_valid_selection:
    try:
        skip_global_load = False
        curr_active_pipeline = active_session_pickle_file_widget.on_load_global(curr_active_pipeline=curr_active_pipeline, basedir=basedir, extended_computations_include_includelist=extended_computations_include_includelist, force_recompute_override_computations_includelist=force_recompute_override_computations_includelist,
                                    skip_global_load=skip_global_load, force_reload=False, override_global_computation_results_pickle_path=active_session_pickle_file_widget.active_global_pkl)
        # Update the global variable after loading global
        print(f'on_load_global(...) complete. workspace variables updated: curr_active_pipeline, custom_suffix, proposed_load_pkl_path')
    except Exception as e:
        print(f'encountered exception loading global e: {e}.')
        pass
        # raise e



## From `test_non_interactive_crash.py`

In [10]:
### Bapun Open-Field Experiment (2022-08-09 Analysis)
from neuropy.core.session.SessionSelectionAndFiltering import build_custom_epochs_filters # used particularly to build Bapun-style filters

active_data_mode_name = 'bapun'
# active_data_mode_name = 'rachel'
print(f'active_data_session_types_registered_classes_dict: {active_data_session_types_registered_classes_dict}')
active_data_mode_registered_class = active_data_session_types_registered_classes_dict[active_data_mode_name]
active_data_mode_type_properties = known_data_session_type_properties_dict[active_data_mode_name]


# basedir = Path('/media/halechr/MAX/Data/Rachel/Cho_241117_Session2').resolve()
## INPUTS: basedir 

force_reload = force_reload #True
print(f'force_reload: {force_reload}')
curr_active_pipeline = NeuropyPipeline.try_init_from_saved_pickle_or_reload_if_needed(active_data_mode_name, active_data_mode_type_properties, override_basepath=Path(basedir), force_reload=force_reload) # , override_parameters_flat_keypaths_dict=override_parameters

# _test_session = RachelDataSessionFormat.build_session(Path(r'R:\data\Rachel\merged_M1_20211123_raw_phy'))
# _test_session, loaded_file_record_list = RachelDataSessionFormat.load_session(_test_session)
# _test_session

## ~20m

active_data_session_types_registered_classes_dict: {'bapun': <class 'neuropy.core.session.Formats.Specific.BapunDataSessionFormat.BapunDataSessionFormatRegisteredClass'>, 'kdiba': <class 'neuropy.core.session.Formats.Specific.KDibaOldDataSessionFormat.KDibaOldDataSessionFormatRegisteredClass'>}
force_reload: False
Computing loaded session pickle file results : "W:/Data/Bapun/RatN/Day4OpenField/loadedSessPickle.pkl"... 	done.
	done.
	done.
	done.
	done.
	done.
	done.
	done.
	done.
	done.
	done.
	done.
	done.
	done.
	done.
	done.
build_logger(full_logger_string="2025-11-03_10-11-00.Apogee.bapun.RatN.Day4OpenField", file_logging_dir: None):
done.
Loading pickled pipeline success: W:\Data\Bapun\RatN\Day4OpenField\loadedSessPickle.pkl.


	 time variable changed from 't_rel_seconds' to 't_seconds'.


	 time variable changed!
properties already present in pickled version. No need to save.
pipeline load success!


##### Old not needed anymore manual comps

In [ ]:
curr_epoch_names: List[str] = curr_active_pipeline.sess.epochs.to_dataframe()['label'].to_list()
print(f'curr_epoch_names: {curr_epoch_names}')

In [11]:
from neuropy.core.session.SessionSelectionAndFiltering import build_custom_epochs_filters

# epoch_name_includelist = ['pre', 'maze1', 'post1', 'maze2', 'post2']
# epoch_name_includelist = ['pre', 'roam', 'sprinkle', 'post']
# epoch_name_includelist = ['roam', 'sprinkle']

# active_session_filter_configurations = build_custom_epochs_filters(curr_active_pipeline.sess, epoch_name_includelist=['pre', 'maze1', 'post1', 'maze2', 'post2']) ## ALL possible epochs

# active_session_filter_configurations = build_custom_epochs_filters(curr_active_pipeline.sess, epoch_name_includelist=['maze1', 'maze2', 'maze_GLOBAL']) ## ALL possible epochs
active_session_filter_configurations = build_custom_epochs_filters(curr_active_pipeline.sess, epoch_name_includelist=['maze1', 'maze2', 'maze_GLOBAL']) ## ALL possible epochs

# active_session_filter_configurations = active_data_mode_registered_class.build_default_filter_functions(sess=curr_active_pipeline.sess)
# active_session_filter_configurations = build_custom_epochs_filters(curr_active_pipeline.sess, epoch_name_includelist=['pre', 'roam', 'maze', 'sprinkle', 'post']) ## ALL possible epochs
# active_session_filter_configurations = build_custom_epochs_filters(curr_active_pipeline.sess, epoch_name_includelist=['pre', 'roam', 'sprinkle', 'post']) ## ALL possible epochs


# active_session_filter_configurations = active_data_mode_registered_class.build_default_filter_functions(sess=curr_active_pipeline.sess, epoch_name_includelist=epoch_name_includelist) # build_filters_pyramidal_epochs(sess=curr_kdiba_pipeline.sess)
# active_session_filter_configurations = build_custom_epochs_filters(curr_active_pipeline.sess, epoch_name_includelist=['maze','sprinkle'])
# active_session_filter_configurations = build_custom_epochs_filters(curr_active_pipeline.sess, epoch_name_includelist=['maze', 'sprinkle'])
# active_session_filter_configurations = build_custom_epochs_filters(curr_active_pipeline.sess, epoch_name_includelist=['roam', 'sprinkle']) # , 'maze'

# active_session_filter_configurations = active_data_mode_registered_class.build_filters_pyramidal_epochs(curr_active_pipeline.sess, epoch_name_includelist=['maze','sprinkle'])
# active_session_filter_configurations


In [12]:
curr_active_pipeline.filter_sessions(active_session_filter_configurations)


In [13]:
active_session_computation_configs = active_data_mode_registered_class.build_active_computation_configs(sess=curr_active_pipeline.sess, time_bin_size=0.5)
active_session_computation_configs

[DynamicContainer({'pf_params': <PlacefieldComputationParameters: {'speed_thresh': 10.0, 'grid_bin': (4.837428803535832, 3.096345720961236), 'grid_bin_bounds': (None, None), 'smooth': (2.0, 2.0), 'frate_thresh': 1.0, 'is_directional': False, 'time_bin_size': 0.5, 'computation_epochs': None};>, 'spike_analysis': DynamicContainer({'max_num_spikes_per_neuron': 20000, 'kleinberg_parameters': DynamicContainer({'s': 2, 'gamma': 0.2}), 'use_progress_bar': False, 'debug_print': False})})]

In [ ]:
active_session_computation_configs[0].pf_params.computation_epochs

In [ ]:
# grid_bin_bounds=(((-83.33747881216672, 110.15967332926644), (-94.89955475226206, 97.07387994733473)))


bapun_open_field_grid_bin_bounds = (((-120.0, 120.0), (-120.0, 120.0)))
curr_active_pipeline.get_all_parameters()
# curr_active_pipeline.update_parameters(grid_bin_bounds = (((-120.0, 120.0), (-120.0, 120.0))))
curr_active_pipeline.sess.config.grid_bin_bounds = (((-120.0, 120.0), (-120.0, 120.0)))


# override_parameters_flat_keypaths_dict = {'grid_bin_bounds': (((-120.0, 120.0), (-120.0, 120.0))), # 'rank_order_shuffle_analysis.minimum_inclusion_fr_Hz': minimum_inclusion_fr_Hz,
# 										#   'sess.config.preprocessing_parameters.laps.use_direction_dependent_laps': False, # lap_estimation_parameters
#                                         }

# curr_active_pipeline.update_parameters(override_parameters_flat_keypaths_dict=override_parameters_flat_keypaths_dict) # should already be updated, but try it again anyway.


In [ ]:
from neuropy.core.epoch import Epoch, EpochsAccessor, ensure_dataframe, ensure_Epoch


# ==================================================================================================================================================================================================================================================================================== #
# Update computation_epochs to be only the maze ones                                                                                                                                                                                                                                   #
# ==================================================================================================================================================================================================================================================================================== #

## activity_only_epochs_df:
epochs_df = ensure_dataframe(deepcopy(curr_active_pipeline.sess.epochs))
# activity_only_epochs_df: pd.DataFrame = epochs_df[epochs_df['label'].isin(['maze1', 'maze2', 'maze_GLOBAL'])]

activity_only_epochs_df: pd.DataFrame = epochs_df[epochs_df['label'].isin(['maze1', 'maze2'])].epochs.get_non_overlapping_df()
activity_only_epochs: Epoch = ensure_Epoch(activity_only_epochs_df, metadata=curr_active_pipeline.sess.epochs.metadata)

## GLobal only ('maze_GLOBAL')
epochs_df = ensure_dataframe(deepcopy(curr_active_pipeline.sess.epochs))
global_activity_only_epochs_df: pd.DataFrame = epochs_df[epochs_df['label'].isin(['maze_GLOBAL'])].epochs.get_non_overlapping_df()
global_activity_only_epoch: Epoch = ensure_Epoch(global_activity_only_epochs_df, metadata=curr_active_pipeline.sess.epochs.metadata)

## OUTPUTS: activity_only_epochs, global_activity_only_epoch

## OUTPUTS: activity_only_epoch


# active_session_computation_configs[0].pf_params.computation_epochs = deepcopy(curr_active_pipeline.filtered_sessions['maze'].epochs)
# active_session_computation_configs[0].pf_params.computation_epochs = deepcopy(curr_active_pipeline.sess.epochs)
# active_session_computation_configs[0].pf_params.computation_epochs = deepcopy(curr_active_pipeline.sess.epochs) ## prev
active_session_computation_configs[0].pf_params.computation_epochs = deepcopy(activity_only_epochs)

global_only_sess_comp_config = deepcopy(active_session_computation_configs[0])
global_only_sess_comp_config.pf_params.computation_epochs = deepcopy(global_activity_only_epoch)
if len(active_session_computation_configs) < 2:
    active_session_computation_configs.append(global_only_sess_comp_config)
else:
    active_session_computation_configs[1] = global_only_sess_comp_config

# active_session_computation_configs[0].pf_params.computation_epochs = deepcopy(bapun_epochs)
# active_session_computation_configs[1].pf_params.computation_epochs = deepcopy(curr_active_pipeline.filtered_sessions['maze'].epochs.to_dataframe())
active_session_computation_configs
# active_session_computation_configs[0].pf_params.computation_epochs

#    start   stop     label  duration
# 0      0   7407       pre      7407
# 1   7423  11483      maze      4060
# 3  10186  11483  sprinkle      1297
# 2  11497  25987      post     14490

# [4 rows x 4 columns]

## UPDATES: active_session_computation_configs


In [14]:
active_session_computation_configs[0].pf_params.linearization_method = "umap"

for an_epoch_name, a_sess in curr_active_pipeline.filtered_sessions.items():
    ## forcibly compute the linearized position so it doesn't fallback to "isomap" method which eats all the memory
    a_pos_df: pd.DataFrame = a_sess.position.compute_linearized_position(method='umap')
    


c:\Users\pho\repos\Spike3DWorkEnv\Spike3D\.venv\lib\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
c:\Users\pho\repos\Spike3DWorkEnv\Spike3D\.venv\lib\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


In [ ]:
# activity_only_epoch_names: List[str] = ['maze1', 'maze2', 'maze_GLOBAL']
# active_computation_functions_name_includelist
activity_only_epoch_names: List[str] = active_session_computation_configs[0].pf_params.computation_epochs.labels.tolist() ## should be same as config
activity_only_epoch_names

# # Create non-overlapping version
# non_overlapping_epochs = ensure_Epoch(active_session_computation_configs[0].pf_params.computation_epochs.epochs.get_non_overlapping_df())
# active_session_computation_configs[0].pf_params.computation_epochs = non_overlapping_epochs


In [ ]:
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import NeuropyPipeline
from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_extended_computations

curr_active_pipeline.reload_default_computation_functions()
    
active_computation_functions_name_includelist = ['pf_computation',
                                                'pfdt_computation',
                                                'position_decoding',
                                                #  'position_decoding_two_step',
                                                #  'extended_pf_peak_information',
                                                ] # 'ratemap_peaks_prominence2d'


## Loops through all configs
for i, a_config in enumerate(active_session_computation_configs):
    active_epoch_names: List[str] = a_config.pf_params.computation_epochs.labels.tolist() ## should be same as config
    print(f'i: {i}, active_epoch_names: {active_epoch_names}') # (activity_only_epoch_names)

    # curr_active_pipeline.perform_computations(active_session_computation_configs[0], computation_functions_name_excludelist=['_perform_spike_burst_detection_computation', '_perform_velocity_vs_pf_density_computation', '_perform_velocity_vs_pf_simplified_count_density_computation']) # SpikeAnalysisComputations._perform_spike_burst_detection_computation
    # curr_active_pipeline.perform_computations(active_session_computation_configs[0], computation_functions_name_includelist=active_computation_functions_name_includelist, enabled_filter_names=activity_only_epoch_names, overwrite_extant_results=True, fail_on_exception=False, debug_print=True) # SpikeAnalysisComputations._perform_spike_burst_detection_computation
    curr_active_pipeline.perform_computations(a_config, computation_functions_name_includelist=active_computation_functions_name_includelist, enabled_filter_names=active_epoch_names, overwrite_extant_results=False, fail_on_exception=False, debug_print=True) # SpikeAnalysisComputations._perform_spike_burst_detection_computation



In [ ]:
curr_active_pipeline.active_completed_computation_result_names

In [ ]:
# curr_active_pipeline.perform_computations(active_session_computation_configs[0], computation_functions_name_includelist=active_computation_functions_name_includelist, enabled_filter_names=['maze1', 'maze2'], overwrite_extant_results=False, fail_on_exception=False, debug_print=True)

In [15]:
# curr_active_pipeline.computation_results['maze'].accumulated_errors
curr_active_pipeline.clear_all_failed_computations()

{}

In [16]:
curr_active_pipeline.prepare_for_display(root_output_dir=r'Output', should_smooth_maze=True) # TODO: pass a display config
# curr_active_pipeline.prepare_for_display(root_output_dir=r'W:\Data\Output', should_smooth_maze=True) # TODO: pass a display config

In [ ]:
curr_active_pipeline.pickle_path
curr_active_pipeline.global_computation_results_pickle_path
curr_active_pipeline.get_output_path()

In [17]:
# _out = curr_active_pipeline.save_pipeline(saving_mode=PipelineSavingScheme.TEMP_THEN_OVERWRITE)
_out = curr_active_pipeline.save_pipeline(saving_mode=PipelineSavingScheme.OVERWRITE_IN_PLACE)
# _out = curr_active_pipeline.save_pipeline(saving_mode=PipelineSavingScheme.TEMP_THEN_OVERWRITE, active_pickle_filename='loadedSessPickle_2025-02-26.pkl')


finalized_loaded_sess_pickle_path: W:\Data\Bapun\RatN\Day4OpenField\loadedSessPickle.pkl


Saving (file mode 'w+b') pickle file results : "W:/Data/Bapun/RatN/Day4OpenField/loadedSessPickle.pkl"... 	moving new output at 'W:\Data\Bapun\RatN\Day4OpenField\20251103101417-loadedSessPickle.pkltmp' -> to desired location: 'W:\Data\Bapun\RatN\Day4OpenField\loadedSessPickle.pkl'
	Saved file size: 3482.68 MB
saved pickle file


In [18]:
_out = curr_active_pipeline.save_global_computation_results()#save_pipeline(saving_mode=PipelineSavingScheme.TEMP_THEN_OVERWRITE, active_pickle_filename='loadedSessPickle_2025-02-27.pkl')

global_computation_results_pickle_path: W:\Data\Bapun\RatN\Day4OpenField\output\global_computation_results.pkl
Saving (file mode 'w+b') pickle file results : "W:/Data/Bapun/RatN/Day4OpenField/output/global_computation_results.pkl"... 	moving new output at 'W:\Data\Bapun\RatN\Day4OpenField\output\20251103101438-global_computation_results.pkltmp' -> to desired location: 'W:\Data\Bapun\RatN\Day4OpenField\output\global_computation_results.pkl'
	Saved file size: 2083.32 MB
saved pickle file


In [19]:
# include_includelist = ['pre', 'maze1', 'post1', 'maze2', 'post2', 'maze',]
include_includelist = ['roam', 'sprinkle']
# include_includelist = curr_active_pipeline.filtered_session_names
include_includelist

['roam', 'sprinkle']

In [ ]:
curr_active_pipeline.filtered_session_names

In [ ]:
include_includelist = curr_active_pipeline.filtered_session_names
print(f'include_includelist: {include_includelist}')

In [20]:
## Setup Computation Functions to be executed:
# includelist Mode:
computation_functions_name_includelist=['_perform_baseline_placefield_computation', '_perform_time_dependent_placefield_computation', '_perform_extended_statistics_computation',
                                '_perform_position_decoding_computation', 
                                '_perform_firing_rate_trends_computation',
                                '_perform_pf_find_ratemap_peaks_computation',
                                # '_perform_time_dependent_pf_sequential_surprise_computation'
                                '_perform_two_step_position_decoding_computation',
                                # '_perform_recursive_latent_placefield_decoding'
                            ]  # '_perform_pf_find_ratemap_peaks_peak_prominence2d_computation'
computation_functions_name_excludelist=None

batch_extended_computations(curr_active_pipeline, included_computation_filter_names=computation_functions_name_includelist, include_includelist=include_includelist,
                            include_global_functions=True, fail_on_exception=False, progress_print=True, debug_print=False)


included includelist is specified: ['roam', 'sprinkle'], so only performing these extended computations.
Running batch_extended_computations(...) with included_computation_filter_names: "['_perform_baseline_placefield_computation', '_perform_time_dependent_placefield_computation', '_perform_extended_statistics_computation', '_perform_position_decoding_computation', '_perform_firing_rate_trends_computation', '_perform_pf_find_ratemap_peaks_computation', '_perform_two_step_position_decoding_computation']"
done with all batch_extended_computations(...).


[]

## NEW BATCH COMPUTE ALL

In [21]:
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import final_process_bapun_all_comps

# try:
curr_active_pipeline = final_process_bapun_all_comps(curr_active_pipeline=curr_active_pipeline, posthoc_save=False)
# except Exception as e:
#     print(f'exception: {e}')
#     # raise e
#     pass    

## 9m


hardcoded_params.decoder_building_session_names: ['roam', 'sprinkle', 'maze_GLOBAL']
hardcoded_params.non_global_activity_session_names: ['roam', 'sprinkle']
active_data_session_types_registered_classes_dict: {'bapun': <class 'neuropy.core.session.Formats.Specific.BapunDataSessionFormat.BapunDataSessionFormatRegisteredClass'>, 'kdiba': <class 'neuropy.core.session.Formats.Specific.KDibaOldDataSessionFormat.KDibaOldDataSessionFormatRegisteredClass'>}
WARN: already fixedup session epochs.
curr_epoch_names: ['pre', 'roam', 'sprinkle', 'post', 'maze']


c:\Users\pho\repos\Spike3DWorkEnv\Spike3D\.venv\lib\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
c:\Users\pho\repos\Spike3DWorkEnv\Spike3D\.venv\lib\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


i: 0, active_epoch_names: ['roam', 'sprinkle']
Performing perform_action_for_all_contexts with action EvaluationActions.EVALUATE_COMPUTATIONS on filtered_session with filter named "roam"...
curr_active_computation_params.pf_params.computation_epochs: 1 epochs
array([[7423, 10185]])

	 TODO: this will prevent recomputation even when the excludelist/includelist or computation function definitions change. Rework so that this is smarter.
updating computation_results...
done.
Performing perform_action_for_all_contexts with action EvaluationActions.EVALUATE_COMPUTATIONS on filtered_session with filter named "sprinkle"...
curr_active_computation_params.pf_params.computation_epochs: 1 epochs
array([[10186, 11483]])

	 TODO: this will prevent recomputation even when the excludelist/includelist or computation function definitions change. Rework so that this is smarter.
updating computation_results...
done.
i: 1, active_epoch_names: []
updating computation_results...
done.
updating computation_re

# 2025-09-08 - Concatenate Movement Sessions

## Decode across both movement sessions

In [ ]:
list(curr_active_pipeline.filtered_sessions.keys())


In [39]:
from neuropy.core.session.Formats.Specific.BapunDataSessionFormat import BapunDataSessionFormatRegisteredClass
from neuropy.core.epoch import Epoch, ensure_dataframe, ensure_Epoch, EpochsAccessor
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import build_contextual_pf2D_decoder, decode_using_contextual_pf2D_decoder
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import decode_using_contextual_pf2D_decoder
from pyphoplacecellanalysis.Analysis.Decoder.reconstruction import DecodedFilterEpochsResult, SingleEpochDecodedResult
from neuropy.core.session.Formats.BaseDataSessionFormats import HardcodedProcessingParameters

## Build the merged decoder `contextual_pf2D`
# ACTUALLY BUILD THE PSEUDO 2D: ______________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________ #
hardcoded_params: HardcodedProcessingParameters = BapunDataSessionFormatRegisteredClass._get_session_specific_parameters(session_context=curr_active_pipeline.get_session_context())
epochs_to_create_global_from_names = hardcoded_params.non_global_activity_session_names # ['maze1', 'maze2'] or ['roam', 'sprinkle']
pf2D_Decoder_dict, contextual_pf2D, contextual_pf2D_Decoder = build_contextual_pf2D_decoder(curr_active_pipeline, epochs_to_create_global_from_names=epochs_to_create_global_from_names)


self will be re-binned to match target_pf...
done.
k: sprinkle: did_update_bins: True


In [40]:
# active_laps_decoding_time_bin_size: float = 0.25
# active_laps_decoding_time_bin_size: float = 1.0
active_laps_decoding_time_bin_size: float = 0.25

all_context_filter_epochs_decoder_result, global_only_epoch = decode_using_contextual_pf2D_decoder(curr_active_pipeline, contextual_pf2D_Decoder=contextual_pf2D_Decoder, active_laps_decoding_time_bin_size=active_laps_decoding_time_bin_size)
# 10m 2s

## 32m at 0.25

epochs_to_merge_as_global_epoch_names: ['pre', 'roam', 'sprinkle', 'post', 'maze']


C:\Users\pho\repos\Spike3DWorkEnv\pyPhoPlaceCellAnalysis\src\pyphoplacecellanalysis\Analysis\Decoder\reconstruction.py:2616: RuntimeWarning: invalid value encountered in divide
  curr_unit_marginal_x.p_x_given_n = curr_unit_marginal_x.p_x_given_n / np.sum(curr_unit_marginal_x.p_x_given_n, axis=0) # sum over all positions for each time_bin (so there's a normalized distribution at each timestep)
C:\Users\pho\repos\Spike3DWorkEnv\pyPhoPlaceCellAnalysis\src\pyphoplacecellanalysis\Analysis\Decoder\reconstruction.py:2628: RuntimeWarning: invalid value encountered in divide
  curr_unit_marginal_y.p_x_given_n = curr_unit_marginal_y.p_x_given_n / np.sum(curr_unit_marginal_y.p_x_given_n, axis=0) # sum over all positions for each time_bin (so there's a normalized distribution at each timestep)


#### Build `DirectionalDecodersContinuouslyDecodedResult` object to hold all the decoded epochs/decoders/etc:

In [41]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.DirectionalPlacefieldGlobalComputationFunctions import DirectionalDecodersContinuouslyDecodedResult

global_spikes_df: pd.DataFrame = deepcopy(curr_active_pipeline.sess.spikes_df)
directional_decoders_decode_result: DirectionalDecodersContinuouslyDecodedResult = DirectionalDecodersContinuouslyDecodedResult(pf1D_Decoder_dict=pf2D_Decoder_dict, pseudo2D_decoder=contextual_pf2D_Decoder, spikes_df=global_spikes_df, continuously_decoded_result_cache_dict={
        active_laps_decoding_time_bin_size: dict(pseudo2D = all_context_filter_epochs_decoder_result,
                                        #     **individual_decoder_decoding_results,
                                            ),
	})
curr_active_pipeline.global_computation_results.computed_data['DirectionalDecodersDecoded'] = directional_decoders_decode_result

In [ ]:
directional_decoders_decode_result_pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2025-09-23_directional_decoders_decode_result.pkl')


In [ ]:
## Load from pickle
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.DirectionalPlacefieldGlobalComputationFunctions import DirectionalDecodersContinuouslyDecodedResult

directional_decoders_decode_result: DirectionalDecodersContinuouslyDecodedResult = DirectionalDecodersContinuouslyDecodedResult.from_file(pkl_path=directional_decoders_decode_result_pkl_output_path)
curr_active_pipeline.global_computation_results.computed_data['DirectionalDecodersDecoded'] = directional_decoders_decode_result ## assign to curr_active_pipeline's global result
directional_decoders_decode_result

In [ ]:
directional_decoders_decode_result_pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2025-09-23_directional_decoders_decode_result.pkl')
directional_decoders_decode_result.save(pkl_output_path=directional_decoders_decode_result_pkl_output_path)
print(f'directional_decoders_decode_result_pkl_output_path: "{directional_decoders_decode_result_pkl_output_path.as_posix()}"')

### OLD

#### Don't need this in the end

In [ ]:

# from pyphoplacecellanalysis.Analysis.Decoder.reconstruction import BayesianPlacemapPositionDecoder
# active_laps_decoding_time_bin_size = 1.0

## INPUTS: pf2D_Decoder_dict, 
desired_global_created_epoch_name: str = 'maze_GLOBAL'
epochs_to_decode_dict = {desired_global_created_epoch_name: deepcopy(global_only_epoch)}
individual_decoder_decoding_results: Dict[str, SingleEpochDecodedResult] = {}

for a_decoder_name, a_decoder in pf2D_Decoder_dict.items():    
    global_spikes_df: pd.DataFrame = deepcopy(curr_active_pipeline.sess.spikes_df)
    individual_decoder_decoding_results[a_decoder_name] = a_decoder.decode_specific_epochs(spikes_df=deepcopy(global_spikes_df), filter_epochs=ensure_dataframe(epochs_to_decode_dict[desired_global_created_epoch_name]), decoding_time_bin_size=active_laps_decoding_time_bin_size, debug_print=False)    
    individual_decoder_decoding_results[a_decoder_name].marginal_z_list = [None]
    
    # ## ASSIGN IT to .marginal_z
    # individual_decoder_decoding_results[a_decoder_name].marginal_z_list = [] ## empty
    # for p_x_given_n in individual_decoder_decoding_results[a_decoder_name].p_x_given_n_list:
    #     n_x_bins, n_y_bins, n_contexts, n_t_bins = np.shape(p_x_given_n) # (41, 63, 2, 51974)
    #     marginal_z = np.nansum(p_x_given_n, axis=(0, 1)) 
    #     marginal_z = marginal_z / np.sum(marginal_z, axis=0, keepdims=True) # sum over all directions for each time_bin (so there's a normalized distribution at each timestep)
    #     individual_decoder_decoding_results[a_decoder_name].marginal_z_list.append(DynamicContainer(p_x_given_n=marginal_z, most_likely_positions_2D=None))

    # try:
    #     individual_decoder_decoding_results[a_decoder_name] = individual_decoder_decoding_results[a_decoder_name].get_result_for_epoch(0)
    # except Exception as e:
    #     print(f'encountered exception: {e}')
    #     pass
    #     # raise e

#.get_result_for_epoch(0)
    
## 4m
    

In [ ]:
## Post-hoc (one time hopefully) fixup:
for a_decoder_name, a_decoder_result in individual_decoder_decoding_results.items():    

    ## ASSIGN IT to .marginal_z
    a_decoder_result.marginal_z_list = [] ## empty
    for p_x_given_n in a_decoder_result.p_x_given_n_list:
        n_x_bins, n_y_bins, n_contexts, n_t_bins = np.shape(p_x_given_n) # (41, 63, 2, 51974)
        marginal_z = np.nansum(p_x_given_n, axis=(0, 1)) 
        marginal_z = marginal_z / np.sum(marginal_z, axis=0, keepdims=True) # sum over all directions for each time_bin (so there's a normalized distribution at each timestep)
        a_decoder_result.marginal_z_list.append(DynamicContainer(p_x_given_n=marginal_z, most_likely_positions_2D=None))

    individual_decoder_decoding_results[a_decoder_name] = a_decoder_result.get_result_for_epoch(0)

In [ ]:
from pyphoplacecellanalysis.Analysis.Decoder.reconstruction import DecodedFilterEpochsResult, SingleEpochDecodedResult

In [ ]:
## ADD CORRECT CONTEXT DECODING MARGINALS:

## INPUTS: all_context_filter_epochs_decoder_result
# all_context_filter_epochs_decoder_result.marginal_z
p_x_given_n = all_context_filter_epochs_decoder_result.p_x_given_n
n_x_bins, n_y_bins, n_contexts, n_t_bins = np.shape(p_x_given_n) # (41, 63, 2, 51974)

marginal_z = np.nansum(p_x_given_n, axis=(0, 1)) 
marginal_z = marginal_z / np.sum(marginal_z, axis=0, keepdims=True) # sum over all directions for each time_bin (so there's a normalized distribution at each timestep)
# marginal_z
print(f'marginal_z.shape: {np.shape(marginal_z)}')

# most_likely_context_idx = np.argmax(marginal_z, axis=0)
# n_x_bins, n_y_bins, n_contexts, n_t_bins = np.shape(p_x_given_n) # (41, 63, 2, 51974)

# most_liekly_context_only_p_x_given_n = np.array([np.squeeze(p_x_given_n[:, :, a_most_likely_ctxt, t]) for t, a_most_likely_ctxt in enumerate(most_likely_context_idx)])
# np.shape(most_liekly_context_only_p_x_given_n)
# most_liekly_context_only_p_x_given_n = np.where(most_likely_context_idx, np.squeeze(p_x_given_n[:, :, 1, :]), np.squeeze(p_x_given_n[:, :, 0, :]))
# np.shape(most_liekly_context_only_p_x_given_n) # (41, 63, 51974)

## ASSIGN IT to .marginal_z
all_context_filter_epochs_decoder_result.marginal_z = DynamicContainer(p_x_given_n=marginal_z, most_likely_positions_2D=None)
# all_context_filter_epochs_decoder_result.marginal_x

In [ ]:
marginal_z: NDArray = all_context_filter_epochs_decoder_result.marginal_z.p_x_given_n


### Unpacking a result:

In [ ]:
from pyphoplacecellanalysis.Analysis.Decoder.reconstruction import BasePositionDecoder, DecodedFilterEpochsResult, SingleEpochDecodedResult

## Uses the `global_computation_results.computed_data['DirectionalDecodersDecoded']`
directional_decoders_decode_result: DirectionalDecodersContinuouslyDecodedResult = curr_active_pipeline.global_computation_results.computed_data['DirectionalDecodersDecoded']
# all_directional_pf1D_Decoder_dict: Dict[str, BasePositionDecoder] = directional_decoders_decode_result.pf1D_Decoder_dict
pseudo3D_decoder: BasePositionDecoder = directional_decoders_decode_result.pseudo2D_decoder
# all_directional_pf1D_Decoder_dict: Dict[str, BasePositionDecoder] = directional_decoders_decode_result.pf1D_Decoder_dict
continuously_decoded_result_cache_dict = directional_decoders_decode_result.continuously_decoded_result_cache_dict
continuously_decoded_pseudo2D_decoder_dict = directional_decoders_decode_result.continuously_decoded_pseudo2D_decoder_dict
# continuously_decoded_result_cache_dict

## Unpacking a result:
all_context_filter_epochs_decoder_result: SingleEpochDecodedResult = list(continuously_decoded_pseudo2D_decoder_dict.values())[-1]
marginal_z: NDArray = all_context_filter_epochs_decoder_result.marginal_z.p_x_given_n

In [ ]:
## NOT goinng to work at all for this
# epochs_decoding_time_bin_size: float = 0.50 # half second
# a_new_fully_generic_result: GenericDecoderDictDecodedEpochsDictResult = GenericDecoderDictDecodedEpochsDictResult.batch_user_compute_fn(curr_active_pipeline=curr_active_pipeline, force_recompute=True, time_bin_size=epochs_decoding_time_bin_size, debug_print=True)
# # valid_EpochComputations_result.a_generic_decoder_dict_decoded_epochs_dict_result = a_new_fully_generic_result

## OUTPUTS: contextual_pf2D_dict, contextual_pf2D, contextual_pf2D_Decoder, all_context_filter_epochs_decoder_result, global_only_epoch
all_context_filter_epochs_decoder_result.marginal_x = 


In [ ]:
_out = dict()
_out['_display_spike_rasters_window'] = curr_active_pipeline.display(display_function='_display_spike_rasters_window', active_session_configuration_context=curr_active_pipeline.get_session_context().adding_context_if_missing(filter_name='maze_GLOBAL')) # _display_spike_rasters_window


In [ ]:
## For Day4OpenField type sessions
epochs_df = ensure_dataframe(curr_active_pipeline.sess.epochs)
epochs_df = epochs_df.epochs.adding_concatenated_epoch(epochs_to_create_global_from_names=['pre', 'roam', 'sprinkle', 'post'], created_epoch_name='maze_any')
global_only_epoch: Epoch = ensure_Epoch(epochs_df[(epochs_df['label'] == 'maze_any')])
# global_only_epoch


active_laps_decoding_time_bin_size = 0.250
contextual_pf2D_dict, contextual_pf2D, contextual_pf2D_Decoder, all_context_filter_epochs_decoder_result = build_contextual_pf2D_decoder(curr_active_pipeline, epochs_to_create_global_from_names = ['roam', 'sprinkle'], active_laps_decoding_time_bin_size=0.75)
## OUTPUTS: contextual_pf2D_Decoder


In [ ]:
## INPUTS: contextual_pf2D_Decoder
global_decoded_cache = {}
# active_laps_decoding_time_bin_sizes = [0.050, 0.1, 0.25, 1.0]
# active_laps_decoding_time_bin_sizes = [0.1, 1.0]
active_laps_decoding_time_bin_sizes = [0.1,]
for active_time_bin_size in active_laps_decoding_time_bin_sizes:
    an_all_context_filter_epochs_decoder_result, global_only_epoch = decode_using_contextual_pf2D_decoder(curr_active_pipeline, contextual_pf2D_Decoder=contextual_pf2D_Decoder, active_laps_decoding_time_bin_size=active_time_bin_size)	
    global_decoded_cache[active_time_bin_size] = deepcopy(an_all_context_filter_epochs_decoder_result)

## OUTPUTS: global_only_epoch


In [ ]:
from pyphoplacecellanalysis.Analysis.Decoder.reconstruction import DecodedFilterEpochsResult, SingleEpochDecodedResult
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import DecodeSpecificEpochsResultWithDecodingInfo

## INPUTS: contextual_pf2D_Decoder, curr_active_pipeline
## Decode PBEs please
pbes = deepcopy(curr_active_pipeline.sess.pbe)
# ripple_decoding_time_bin_size: float = 0.025 # 25ms
ripple_decoding_time_bin_size: float = 0.060 # 25ms
global_spikes_df: pd.DataFrame = deepcopy(curr_active_pipeline.sess.spikes_df)
pbes_full_result: DecodeSpecificEpochsResultWithDecodingInfo = DecodeSpecificEpochsResultWithDecodingInfo.init_by_decoding(decoding_context=IdentifyingContext(epoch_name='pbe'),
																														   decoder=contextual_pf2D_Decoder, spikes_df=deepcopy(global_spikes_df), filter_epochs=ensure_dataframe(pbes), decoding_time_bin_size=ripple_decoding_time_bin_size)


## 18m at 60ms

## OUTPUTS: pbes, ripple_decoding_time_bin_size,  global_spikes_df, pbe_decoder_result

In [ ]:
pbes_full_result_pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2025-09-23_pbes_full_result.pkl')
pbes_full_result.save(pkl_output_path=pbes_full_result_pkl_output_path)
print(f'pbes_full_result_pkl_output_path: "{pbes_full_result_pkl_output_path.as_posix()}"')



In [ ]:
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import DecodeSpecificEpochsResultWithDecodingInfo

pbes_full_result_pkl_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2025-09-23_pbes_full_result.pkl')

pbes_full_result: DecodeSpecificEpochsResultWithDecodingInfo = DecodeSpecificEpochsResultWithDecodingInfo.load(pbes_full_result_pkl_output_path)
pbes_full_result

In [ ]:
# pbes_full_result_h5_output_path: Path = curr_active_pipeline.get_output_path().joinpath('2025-09-23_pbes_full_result.h5')
# pbes_full_result.to_hdf(file_path=pbes_full_result_h5_output_path, key='pbes_full_result') # , enable_hdf_testing_mode=True, OVERRIDE_ALLOW_GLOBAL_NESTED_EXPANSION=True
# print(f'pbes_full_result_h5_output_path: "{pbes_full_result_h5_output_path.as_posix()}"')
# # _ALLOW_GLOBAL_NESTED_EXPANSION



# 💾 Save Export Pipeline


In [ ]:
curr_active_pipeline.get_complete_session_context()
custom_save_filepaths, custom_save_filenames, custom_suffix = curr_active_pipeline.get_custom_pipeline_filenames_from_parameters()
custom_save_filenames

In [ ]:
custom_save_filenames['pipeline_pkl']
custom_save_filenames['global_computation_pkl']

pickle_path = 'loadedSessPickle_withNormalComputedReplays-qclu_[1, 2, 4, 6, 7, 9]-frateThresh_5.0_2025-01-20.pkl'
global_computation_pkl = 'global_computation_results_withNormalComputedReplays-qclu_[1, 2, 4, 6, 7, 9]-frateThresh_5.0_2025-01-20.pkl'

In [ ]:
## indicate that it was loaded with a custom suffix
curr_active_pipeline.pickle_path ## correct
curr_active_pipeline.global_computation_results_pickle_path ## correct

# curr_active_pipeline.save_pipeline(saving_mode=PipelineSavingScheme.TEMP_THEN_OVERWRITE, override_pickle_path=curr_active_pipeline.pickle_path, active_pickle_filename=curr_active_pipeline.pickle_path.name) #active_pickle_filename=
# curr_active_pipeline.save_global_computation_results(override_global_pickle_path=curr_active_pipeline.global_computation_results_pickle_path)

In [ ]:
## indicate that it was loaded with a custom suffix
curr_active_pipeline.pickle_path ## correct
curr_active_pipeline.global_computation_results_pickle_path ## correct

if curr_active_pipeline.pickle_path is None:
    active_pickle_filename = 'loadedSessPickle.pkl'
else:
    active_pickle_filename = curr_active_pipeline.pickle_path.name
    
print(f'active_pickle_filename: {active_pickle_filename}')
curr_active_pipeline.save_pipeline(saving_mode=PipelineSavingScheme.TEMP_THEN_OVERWRITE, override_pickle_path=curr_active_pipeline.pickle_path, active_pickle_filename=active_pickle_filename) #active_pickle_filename=


In [ ]:
curr_active_pipeline.save_global_computation_results(override_global_pickle_path=curr_active_pipeline.global_computation_results_pickle_path)

#### Manual Pickling

In [ ]:
# curr_active_pipeline.save_pipeline(saving_mode=PipelineSavingScheme.TEMP_THEN_OVERWRITE, override_pickle_path=curr_active_pipeline.pickle_path, active_pickle_filename='loadedSessPickle_withNormalComputedReplays-qclu_[1, 2, 4, 6, 7, 9]-frateThresh_5.0_2025-01-20.pkl') #active_pickle_filename=
curr_active_pipeline.save_pipeline(saving_mode=PipelineSavingScheme.TEMP_THEN_OVERWRITE, active_pickle_filename='loadedSessPickle_withNormalComputedReplays-qclu_[1, 2, 4, 6, 7, 9]-frateThresh_5.0_2025-01-20.pkl') #active_pickle_filename=

In [ ]:
curr_active_pipeline.save_global_computation_results(override_global_pickle_filename='global_computation_results_withNormalComputedReplays-qclu_[1, 2, 4, 6, 7, 9]-frateThresh_5.0_2025-01-20.pkl')

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.Loading import PipelineWithInputStage, PipelineWithLoadableStage, loadData, saveData

## Custom Save
curr_sess_pkl_path = basedir.joinpath('loadedSessPickle_2025-09-08.pkl')
print(f'saving out to modern pickle: "{curr_sess_pkl_path}"')
saveData(curr_sess_pkl_path, db=curr_active_pipeline, safe_save=True) # (v_dict, str(curr_item_type.__module__), str(curr_item_type.__name__)))


### 2024-06-25 - Load from saved custom

In [ ]:
from pyphocorehelpers.Filesystem.path_helpers import set_posix_windows

# Loads custom pipeline pickles that were saved out via `custom_save_filepaths['pipeline_pkl'] = curr_active_pipeline.save_pipeline(saving_mode=PipelineSavingScheme.TEMP_THEN_OVERWRITE, active_pickle_filename=custom_save_filenames['pipeline_pkl'])`

## INPUTS: global_data_root_parent_path, active_data_mode_name, basedir, saving_mode, force_reload, custom_save_filenames
# custom_suffix: str = '_withNewKamranExportedReplays'

# custom_suffix: str = '_withNewComputedReplays'
# custom_suffix: str = '_withNewComputedReplays-qclu_[1, 2]-frateThresh_5.0'

# custom_save_filenames = {
#     'pipeline_pkl':f'loadedSessPickle{custom_suffix}.pkl',
#     'global_computation_pkl':f"global_computation_results{custom_suffix}.pkl",
#     'pipeline_h5':f'pipeline{custom_suffix}.h5',
# }
# print(f'custom_save_filenames: {custom_save_filenames}')
# custom_save_filepaths = {k:v for k, v in custom_save_filenames.items()}

# # ==================================================================================================================== #
# # PIPELINE LOADING                                                                                                     #
# # ==================================================================================================================== #
# # load the custom saved outputs
# active_pickle_filename = custom_save_filenames['pipeline_pkl'] # 'loadedSessPickle_withParameters.pkl'
# print(f'active_pickle_filename: "{active_pickle_filename}"')
# # assert active_pickle_filename.exists()
# active_session_h5_filename = custom_save_filenames['pipeline_h5'] # 'pipeline_withParameters.h5'
# print(f'active_session_h5_filename: "{active_session_h5_filename}"')

# ==================================================================================================================== #
# Load Pipeline                                                                                                        #
# ==================================================================================================================== #
## DO NOT allow recompute if the file doesn't exist!!
# Computing loaded session pickle file results : "W:/Data/KDIBA/gor01/two/2006-6-07_16-40-19/loadedSessPickle_withNewComputedReplays.pkl"... done.
# Failure loading W:\Data\KDIBA\gor01\two\2006-6-07_16-40-19\loadedSessPickle_withNewComputedReplays.pkl.
# proposed_load_pkl_path = basedir.joinpath(active_pickle_filename).resolve()

## INPUTS: widget.active_global_pkl, widget.active_global_pkl

if active_session_pickle_file_widget.active_global_pkl is None:
    skip_global_load: bool = True
    override_global_computation_results_pickle_path = None
    print(f'skip_global_load: {skip_global_load}')
else:
    skip_global_load: bool = False
    override_global_computation_results_pickle_path = active_session_pickle_file_widget.active_global_pkl.resolve()
    Assert.path_exists(override_global_computation_results_pickle_path)
    print(f'override_global_computation_results_pickle_path: "{override_global_computation_results_pickle_path}"')

proposed_load_pkl_path = active_session_pickle_file_widget.active_local_pkl.resolve()
Assert.path_exists(proposed_load_pkl_path)
proposed_load_pkl_path

custom_suffix: str = active_session_pickle_file_widget.try_extract_custom_suffix()
print(f'custom_suffix: "{custom_suffix}"')

## OUTPUTS: custom_suffix, proposed_load_pkl_path, (override_global_computation_results_pickle_path, skip_global_load)

In [ ]:
epoch_name_includelist

In [ ]:

## INPUTS: proposed_load_pkl_path
# assert proposed_load_pkl_path.exists(), f"for a saved custom the file must exist, but proposed_load_pkl_path: '{proposed_load_pkl_path}' does not!"

epoch_name_includelist=None
# active_computation_functions_name_includelist=['lap_direction_determination', 'pf_computation','firing_rate_trends', 'position_decoding']
active_computation_functions_name_includelist=[]

with set_posix_windows():
    curr_active_pipeline: NeuropyPipeline = batch_load_session(global_data_root_parent_path, active_data_mode_name, basedir, epoch_name_includelist=epoch_name_includelist,
                                            computation_functions_name_includelist=active_computation_functions_name_includelist,
                                            saving_mode=saving_mode, force_reload=force_reload,
                                            skip_extended_batch_computations=True, debug_print=False, fail_on_exception=False, active_pickle_filename=proposed_load_pkl_path, 
                                            override_parameters_flat_keypaths_dict=override_parameters) # , active_pickle_filename = 'loadedSessPickle_withParameters.pkl'

## Post Compute Validate 2023-05-16:
was_updated = BatchSessionCompletionHandler.post_compute_validate(curr_active_pipeline) ## TODO: need to potentially re-save if was_updated. This will fail because constained versions not ran yet.
print(f'Pipeline loaded from custom pickle!!')
## OUTPUT: curr_active_pipeline


In [ ]:
curr_active_pipeline.get_session_context()


In [ ]:
active_data_mode_name = 'bapun'

# curr_active_pipeline.get_session_additional_parameters_context()
# curr_active_pipeline.session_data_type

DataSessionFormatRegistryHolder.get_registry_known_data_session_type_dict()[active_data_mode_name]
DataSessionFormatRegistryHolder.get_registry_data_session_type_class_name_dict()[active_data_mode_name]




In [ ]:
from neuropy.core.session.Formats.BaseDataSessionFormats import HardcodedProcessingParameters
from neuropy.core.session.Formats.Specific.BapunDataSessionFormat import BapunDataSessionFormatRegisteredClass

hardcoded_params: HardcodedProcessingParameters = BapunDataSessionFormatRegisteredClass._get_session_specific_parameters(session_context=curr_active_pipeline.get_session_context())
hardcoded_params

hardcoded_params.decoder_building_session_names
hardcoded_params.non_global_activity_session_names

In [ ]:

@define(slots=False, eq=False, repr=False)
class position_decoding_Parameters(HDF_SerializationMixin, AttrsBasedClassHelperMixin, BaseGlobalComputationParameters):
    """ Docstring for position_decoding_Parameters. 
    """
    override_decoding_time_bin_size: Optional[float] = serialized_attribute_field(default=None)
    ## PARAMS - these are class properties
    override_decoding_time_bin_size_PARAM = param.Number(default=None, doc='override_decoding_time_bin_size param', label='override_decoding_time_bin_size')
    # HDFMixin Conformances ______________________________________________________________________________________________ #
    def to_hdf(self, file_path, key: str, **kwargs):
        """ Saves the object to key in the hdf5 file specified by file_path"""
        super().to_hdf(file_path, key=key, **kwargs)
        



session_specific_parameters: Dict[IdentifyingContext, HardcodedProcessingParameters] = {

    IdentifyingContext(format_name= 'bapun', animal= 'RatN', session_name= 'Day4OpenField'): HardcodedProcessingParameters(decoder_building_session_names=['roam', 'sprinkle', 'maze_GLOBAL'],
																														   ),
																														
    IdentifyingContext(format_name= 'bapun', animal= 'RatN', session_name= 'Day5TwoNovel'): HardcodedProcessingParameters(decoder_building_session_names=['maze1', 'maze2', 'maze_GLOBAL'],
																														   ),
																													
}


In [ ]:
from pyphoplacecellanalysis.General.PipelineParameterClassTemplating import GlobalComputationParametersAttrsClassTemplating

registered_merged_computation_function_default_kwargs_dict, code_str, nested_classes_dict, (imports_dict, imports_list, imports_string) = GlobalComputationParametersAttrsClassTemplating.main_generate_params_classes(curr_active_pipeline=curr_active_pipeline)

code_str

## from `batch_load_session`

In [ ]:
## From `pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing.batch_load_session` 2025-02-26 09:09 
kwargs = {}
epoch_name_includelist=None
# active_computation_functions_name_includelist=['lap_direction_determination', 'pf_computation','firing_rate_trends', 'position_decoding']
active_computation_functions_name_includelist=[]
override_parameters_flat_keypaths_dict = override_parameters
active_pickle_filename = proposed_load_pkl_path
active_session_computation_configs = None
fail_on_exception: bool = False

saving_mode = PipelineSavingScheme.init(saving_mode)
epoch_name_includelist = kwargs.get('epoch_name_includelist', ['maze1','maze2','maze'])
# epoch_name_includelist = ['maze', 'sprinkle']
# epoch_name_includelist = ['pre', 'maze', 'sprinkle', 'post']


debug_print = kwargs.get('debug_print', False)
assert 'skip_save' not in kwargs, f"use saving_mode=PipelineSavingScheme.SKIP_SAVING instead"
# skip_save = kwargs.get('skip_save', False)
# active_pickle_filename = kwargs.get('active_pickle_filename', 'loadedSessPickle.pkl')

# active_session_computation_configs = kwargs.get('active_session_computation_configs', None)
# computation_functions_name_includelist = kwargs.get('computation_functions_name_includelist', None)

known_data_session_type_properties_dict = DataSessionFormatRegistryHolder.get_registry_known_data_session_type_dict(override_parameters_flat_keypaths_dict=override_parameters_flat_keypaths_dict)
active_data_session_types_registered_classes_dict = DataSessionFormatRegistryHolder.get_registry_data_session_type_class_name_dict()

active_data_mode_registered_class = active_data_session_types_registered_classes_dict[active_data_mode_name]
active_data_mode_type_properties = known_data_session_type_properties_dict[active_data_mode_name]

## Begin main run of the pipeline (load or execute):
curr_active_pipeline = NeuropyPipeline.try_init_from_saved_pickle_or_reload_if_needed(active_data_mode_name, active_data_mode_type_properties,
    override_basepath=Path(basedir), force_reload=force_reload, active_pickle_filename=active_pickle_filename, skip_save_on_initial_load=True, override_parameters_flat_keypaths_dict=override_parameters_flat_keypaths_dict)

curr_active_pipeline.update_parameters(override_parameters_flat_keypaths_dict=override_parameters_flat_keypaths_dict) # should already be updated, but try it again anyway.

was_loaded_from_file: bool =  curr_active_pipeline.has_associated_pickle # True if pipeline was loaded from an existing file, False if it was created fresh

# Get the previous configs:
# curr_active_pipeline.filtered_sessions
# ['filtered_session_names', 'filtered_contexts', 'filtered_epochs', 'filtered_sessions']
# loaded_session_filter_configurations = {k:v.filter_config['filter_function'] for k,v in curr_active_pipeline.active_configs.items()}
# loaded_pipeline_computation_configs = {k:v.computation_config for k,v in curr_active_pipeline.active_configs.items()}


## Build updated ones from the current configs:
active_session_filter_configurations = active_data_mode_registered_class.build_default_filter_functions(sess=curr_active_pipeline.sess, epoch_name_includelist=epoch_name_includelist) # build_filters_pyramidal_epochs(sess=curr_kdiba_pipeline.sess)
if debug_print:
    print(f'active_session_filter_configurations: {active_session_filter_configurations}')

## Skip the filtering, it used to be performed bere but NOT NOW

## TODO 2023-05-16 - set `curr_active_pipeline.active_configs[a_name].computation_config.pf_params.computation_epochs = curr_laps_obj` equivalent
## TODO 2023-05-16 - determine appropriate binning from `compute_short_long_constrained_decoders` so it's automatically from the long


In [ ]:
curr_active_pipeline.filtered_session_names

In [ ]:
# epoch_name_includelist = kwargs.get('epoch_name_includelist', ['maze1','maze2','maze'])
# epoch_name_includelist = ['roam', 'sprinkle']
epoch_name_includelist = ['pre', 'roam', 'sprinkle', 'post']
active_session_filter_configurations = active_data_mode_registered_class.build_default_filter_functions(sess=curr_active_pipeline.sess, epoch_name_includelist=epoch_name_includelist) # build_filters_pyramidal_epochs(sess=curr_kdiba_pipeline.sess)


In [ ]:
fail_on_exception: bool = False

In [ ]:
try:
    curr_active_pipeline.save_pipeline(saving_mode=saving_mode, active_pickle_filename=active_pickle_filename, override_pickle_path=kwargs.get('override_pickle_path', None))
except Exception as e:
    exception_info = sys.exc_info()
    an_error = CapturedException(e, exception_info, curr_active_pipeline)
    print(f'WARNING: Failed to save pipeline via `curr_active_pipeline.save_pipeline(...)` with error: {an_error}')
    if fail_on_exception:
        raise

In [ ]:

if active_session_computation_configs is None:
    """
    If there are is provided computation config, get the default:
    """
    # ## Compute shared grid_bin_bounds for all epochs from the global positions:
    # global_unfiltered_session = curr_active_pipeline.sess
    # # ((22.736279243974774, 261.696733348342), (49.989466271998936, 151.2870218547401))
    # first_filtered_session = curr_active_pipeline.filtered_sessions[curr_active_pipeline.filtered_session_names[0]]
    # # ((22.736279243974774, 261.696733348342), (125.5644705153173, 151.21507349463707))
    # second_filtered_session = curr_active_pipeline.filtered_sessions[curr_active_pipeline.filtered_session_names[1]]
    # # ((71.67666779621361, 224.37820920766043), (110.51617463644946, 151.2870218547401))

    # grid_bin_bounding_session = first_filtered_session
    # grid_bin_bounds = PlacefieldComputationParameters.compute_grid_bin_bounds(grid_bin_bounding_session.position.x, grid_bin_bounding_session.position.y)

    ## OR use no grid_bin_bounds meaning they will be determined dynamically for each epoch:
    # grid_bin_bounds = None
    # time_bin_size = 0.03333 #1.0/30.0 # decode at 30fps to match the position sampling frequency
    # time_bin_size = 0.1 # 10 fps
    time_bin_size = kwargs.get('time_bin_size', 0.03333) # 0.03333 = 1.0/30.0 # decode at 30fps to match the position sampling frequency
    # time_bin_size = kwargs.get('time_bin_size', 0.1) # 10 fps

    # lap_estimation_parameters = curr_active_pipeline.sess.config.preprocessing_parameters.epoch_estimation_parameters.laps
    # assert lap_estimation_parameters is not None
    active_session_computation_configs: List[DynamicContainer] = active_data_mode_registered_class.build_active_computation_configs(sess=curr_active_pipeline.sess, time_bin_size=time_bin_size, override_parameters_flat_keypaths_dict=override_parameters_flat_keypaths_dict) # , grid_bin_bounds=grid_bin_bounds

else:
    # Use the provided `active_session_computation_configs`:
    assert 'time_bin_size' not in kwargs, f"time_bin_size kwarg provided but will not be used because a custom active_session_computation_configs was provided as well."

active_session_computation_configs


In [ ]:
# computation_functions_name_includelist = []
computation_functions_name_includelist = ['pf_computation','firing_rate_trends', 'position_decoding']

In [ ]:

## Setup Computation Functions to be executed:
if computation_functions_name_includelist is None:
    # includelist Mode:
    computation_functions_name_includelist=['_perform_baseline_placefield_computation', '_perform_time_dependent_placefield_computation', '_perform_extended_statistics_computation',
                                        '_perform_position_decoding_computation', 
                                        '_perform_firing_rate_trends_computation',
                                        '_perform_pf_find_ratemap_peaks_computation',
                                        # '_perform_time_dependent_pf_sequential_surprise_computation'
                                        # '_perform_two_step_position_decoding_computation',
                                        # '_perform_recursive_latent_placefield_decoding'
                                    ]  # '_perform_pf_find_ratemap_peaks_peak_prominence2d_computation'
    computation_functions_name_excludelist=None
else:
    print(f'using provided computation_functions_name_includelist: {computation_functions_name_includelist}')
    computation_functions_name_excludelist=None

## For every computation config we build a fake (duplicate) filter config).
# OVERRIDE WITH TRUE:
# curr_active_pipeline.sess.config.preprocessing_parameters.epoch_estimation_parameters.laps['use_direction_dependent_laps'] = True # override with True
lap_estimation_parameters = curr_active_pipeline.sess.config.preprocessing_parameters.epoch_estimation_parameters.laps
assert lap_estimation_parameters is not None
use_direction_dependent_laps: bool = lap_estimation_parameters.get('use_direction_dependent_laps', False) # whether to split the laps into left and right directions
# use_direction_dependent_laps: bool = lap_estimation_parameters.get('use_direction_dependent_laps', True) # whether to split the laps into left and right directions

if (use_direction_dependent_laps or (len(active_session_computation_configs) > 3)):
    lap_direction_suffix_list = ['_odd', '_even', '_any'] # ['maze1_odd', 'maze1_even', 'maze1_any', 'maze2_odd', 'maze2_even', 'maze2_any', 'maze_odd', 'maze_even', 'maze_any']
    # lap_direction_suffix_list = ['_odd', '_even', ''] # no '_any' prefix, instead reuses the existing names
    # assert len(lap_direction_suffix_list) == len(active_session_computation_configs), f"len(lap_direction_suffix_list): {len(lap_direction_suffix_list)}, len(active_session_computation_configs): {len(active_session_computation_configs)}, "
else:
    print(f'not using direction-dependent laps.')
    lap_direction_suffix_list = ['']

# active_session_computation_configs: this should contain three configs, one for each Epoch    
active_session_computation_configs = [deepcopy(a_config) for a_config in active_session_computation_configs]

#TODO 2024-10-30 13:22: - [ ] This is where we should override the params using `override_parameters_flat_keypaths_dict`
# if override_parameters_flat_keypaths_dict is not None:
# 	for a_config in active_session_computation_configs:
# 		for k, v in override_parameters_flat_keypaths_dict.items():
# 			try:
# 				a_config.set_by_keypath(k, deepcopy(v))
# 			except Exception as e:
# 				# raise e
# 				print(f'cannot set_by_keypath: {k} -- error: {e}. Skipping for now.')

assert len(lap_direction_suffix_list) == len(active_session_computation_configs)
updated_active_session_pseudo_filter_configs = {} # empty list, woot!


for a_computation_suffix_name, a_computation_config in zip(lap_direction_suffix_list, active_session_computation_configs): # these should NOT be the same length: lap_direction_suffix_list: ['_odd', '_even', '_any']
    # We need to filter and then compute with the appropriate config iteratively.
    for a_filter_config_name, a_filter_config_fn in active_session_filter_configurations.items():
        # TODO: Build a context:
        a_combined_name: str = f'{a_filter_config_name}{a_computation_suffix_name}'
        # if a_computation_suffix_name != '':
        updated_active_session_pseudo_filter_configs[a_combined_name] = deepcopy(a_filter_config_fn) # this copy is just so that the values are recomputed with the appropriate config. This is a HACK
    # end for filter_configs

    ## Actually do the filtering now. We have 
    curr_active_pipeline.filter_sessions(updated_active_session_pseudo_filter_configs, changed_filters_ignore_list=['maze1','maze2','maze'], debug_print=False)

    ## TODO 2023-01-15 - perform_computations for all configs!!
    #TODO 2024-10-30 13:22: - [ ] This is where we should override the params
    # if override_parameters_flat_keypaths_dict is not None:
    # 	for k, v in override_parameters_flat_keypaths_dict.items():
    # 		a_filter_config_fn.set_by_keypath(k, deepcopy(v))

    # if override_parameters_flat_keypaths_dict is not None:
    # 	curr_active_pipeline.update_parameters(override_parameters_flat_keypaths_dict=override_parameters_flat_keypaths_dict) 


    #TODO 2023-10-31 14:58: - [ ] This is where the computations are being done multiple times!
    #TODO 2023-11-13 14:23: - [ ] With this approach, we can't actually properly filter the computation_configs for the relevant sessions ahead of time because they are calculated for a single computation config but across all sessions at once.
    curr_active_pipeline.perform_computations(a_computation_config, computation_functions_name_includelist=computation_functions_name_includelist, computation_functions_name_excludelist=computation_functions_name_excludelist, fail_on_exception=fail_on_exception, debug_print=debug_print) #, overwrite_extant_results=False  ], fail_on_exception=True, debug_print=False)

    if override_parameters_flat_keypaths_dict is not None:
        curr_active_pipeline.update_parameters(override_parameters_flat_keypaths_dict=override_parameters_flat_keypaths_dict) 




In [ ]:
skip_extended_batch_computations = False
fail_on_exception = True
if not skip_extended_batch_computations:
    batch_extended_computations(curr_active_pipeline, include_global_functions=False, fail_on_exception=fail_on_exception, progress_print=True, debug_print=False)
# curr_active_pipeline.perform_computations(active_session_computation_configs[0], computation_functions_name_excludelist=['_perform_spike_burst_detection_computation'], debug_print=False, fail_on_exception=False) # includelist: ['_perform_baseline_placefield_computation']


try:
    curr_active_pipeline.prepare_for_display(root_output_dir=global_data_root_parent_path.joinpath('Output'), should_smooth_maze=True) # TODO: pass a display config
except Exception as e:
    exception_info = sys.exc_info()
    an_error = CapturedException(e, exception_info, curr_active_pipeline)
    print(f'WARNING: Failed to do `curr_active_pipeline.prepare_for_display(...)` with error: {an_error}')
    if fail_on_exception:
        raise

try:
    curr_active_pipeline.save_pipeline(saving_mode=saving_mode, active_pickle_filename=active_pickle_filename, override_pickle_path=kwargs.get('override_pickle_path', None))
except Exception as e:
    exception_info = sys.exc_info()
    an_error = CapturedException(e, exception_info, curr_active_pipeline)
    print(f'WARNING: Failed to save pipeline via `curr_active_pipeline.save_pipeline(...)` with error: {an_error}')
    if fail_on_exception:
        raise

if not saving_mode.shouldSave:
    print(f'saving_mode.shouldSave == False, so not saving at the end of batch_load_session')

## Load pickled global computations:
# If previously pickled global results were saved, they will typically no longer be relevent if the pipeline was recomputed. We need a system of invalidating/versioning the global results when the other computations they depend on change.
# Maybe move into `batch_extended_computations(...)` or integrate with that somehow
# curr_active_pipeline.load_pickled_global_computation_results()


In [ ]:
from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_evaluate_required_computations

skip_global_load = True

In [ ]:
# ==================================================================================================================== #
# Global computations loading:                                                                                            #
# ==================================================================================================================== #
# Loads saved global computations that were saved out via: `custom_save_filepaths['global_computation_pkl'] = curr_active_pipeline.save_global_computation_results(override_global_pickle_filename=custom_save_filenames['global_computation_pkl'])`
## INPUTS: custom_save_filenames
## INPUTS: curr_active_pipeline, (override_global_computation_results_pickle_path, skip_global_load), extended_computations_include_includelist

if skip_global_load:
    override_global_computation_results_pickle_path = None
    print(f'skipping global load because skip_global_load==True')
else:
    # override_global_computation_results_pickle_path = custom_save_filenames['global_computation_pkl']
    print(f'override_global_computation_results_pickle_path: "{override_global_computation_results_pickle_path}"')

# Pre-load ___________________________________________________________________________________________________________ #
force_recompute_global = force_reload
needs_computation_output_dict, valid_computed_results_output_list, remaining_include_function_names = batch_evaluate_required_computations(curr_active_pipeline, include_includelist=extended_computations_include_includelist, include_global_functions=True, fail_on_exception=False, progress_print=True,
                                                    force_recompute=force_recompute_global, force_recompute_override_computations_includelist=force_recompute_override_computations_includelist, debug_print=False)
print(f'Pre-load global computations: needs_computation_output_dict: {[k for k,v in needs_computation_output_dict.items() if (v is not None)]}')
# valid_computed_results_output_list

# Try Unpickling Global Computations to update pipeline ______________________________________________________________ #
if (not force_reload) and (not skip_global_load): # not just force_reload, needs to recompute whenever the computation fails.
    try:
        # INPUTS: override_global_computation_results_pickle_path
        with set_posix_windows():
            sucessfully_updated_keys, successfully_loaded_keys = curr_active_pipeline.load_pickled_global_computation_results(override_global_computation_results_pickle_path=override_global_computation_results_pickle_path,
                                                                                            allow_overwrite_existing=True, allow_overwrite_existing_allow_keys=extended_computations_include_includelist, ) # is new
            print(f'sucessfully_updated_keys: {sucessfully_updated_keys}\nsuccessfully_loaded_keys: {successfully_loaded_keys}')
            did_any_paths_change: bool = curr_active_pipeline.post_load_fixup_sess_basedirs(updated_session_basepath=deepcopy(basedir)) ## use INPUT: basedir
            
    except FileNotFoundError as e:
        exception_info = sys.exc_info()
        e = CapturedException(e, exception_info)
        print(f'cannot load global results because pickle file does not exist! Maybe it has never been created? {e}')
    except Exception as e:
        exception_info = sys.exc_info()
        e = CapturedException(e, exception_info)
        print(f'Unhandled exception: cannot load global results: {e}')
        raise



In [ ]:
print(f'force_reload: {force_reload}, saving_mode: {saving_mode}')
force_reload
saving_mode

In [ ]:
## INPUTS: curr_active_pipeline.global_computation_results_pickle_path, skip_global_load
## indicate that it was loaded with a custom suffix
curr_active_pipeline.pickle_path ## correct
curr_active_pipeline.global_computation_results_pickle_path ## correct

print(f'override_pickle_path = "{curr_active_pipeline.pickle_path}",\nactive_pickle_filename = "{curr_active_pipeline.pickle_path.name}"')
print(f'override_global_pickle_path = "{curr_active_pipeline.global_computation_results_pickle_path}")')

## OUTPUTS: `curr_active_pipeline`  0️⃣ 0️⃣ 0️⃣ 0️⃣ 0️⃣ 0️⃣ 0️⃣ 0️⃣ 0️⃣ 0️⃣ 0️⃣ 0️⃣ 0️⃣ 0️⃣ 0️⃣ 0️⃣ 0️⃣ 0️⃣ 0️⃣ 0️⃣ 0️⃣0️⃣ RESUME Normal Pipeline Load

## 0️⃣ Shared Post-Pipeline load stuff

In [36]:
# BATCH_DATE_TO_USE: str = f'{DAY_DATE_TO_USE}_GL'
# BATCH_DATE_TO_USE: str = f'{DAY_DATE_TO_USE}_rMBP' # TODO: Change this as needed, templating isn't actually doing anything rn.
BATCH_DATE_TO_USE: str = f'{DAY_DATE_TO_USE}_Apogee'
# BATCH_DATE_TO_USE: str = f'{DAY_DATE_TO_USE}_Lab'
 
try:
    if custom_suffix is not None:
        BATCH_DATE_TO_USE = f'{BATCH_DATE_TO_USE}{custom_suffix}'
        print(f'Adding custom suffix: "{custom_suffix}" - BATCH_DATE_TO_USE: "{BATCH_DATE_TO_USE}"')
except NameError as err:
    custom_suffix = None
    print(f'NO CUSTOM SUFFIX.')

known_collected_output_paths = [Path(v).resolve() for v in ['/nfs/turbo/umms-kdiba/Data/Output/collected_outputs', '/home/halechr/FastData/collected_outputs/',
                                                           '/home/halechr/cloud/turbo/Data/Output/collected_outputs',
                                                           r'C:\Users\pho\repos\Spike3DWorkEnv\Spike3D\output\collected_outputs',
                                                           r"K:\scratch\collected_outputs",
                                                           '/Users/pho/data/collected_outputs',
                                                          'output/gen_scripts/']]
collected_outputs_path = find_first_extant_path(known_collected_output_paths)
assert collected_outputs_path.exists(), f"collected_outputs_path: {collected_outputs_path} does not exist! Is the right computer's config commented out above?"
# fullwidth_path_widget(scripts_output_path, file_name_label='Scripts Output Path:')
print(f'collected_outputs_path: {collected_outputs_path}')
# collected_outputs_path.mkdir(exist_ok=True)
# assert collected_outputs_path.exists()

## Build the output prefix from the session context:
active_context = curr_active_pipeline.get_session_context()
curr_session_name: str = curr_active_pipeline.session_name # '2006-6-08_14-26-15'
CURR_BATCH_OUTPUT_PREFIX: str = f"{BATCH_DATE_TO_USE}-{curr_session_name}"
print(f'CURR_BATCH_OUTPUT_PREFIX: "{CURR_BATCH_OUTPUT_PREFIX}"')

collected_outputs_path: L:\repos\Spike3DWorkEnv\Spike3D\output\collected_outputs
CURR_BATCH_OUTPUT_PREFIX: "2025-11-03_Apogee-RatN_Day4_2019-10-15_11-30-06"


# 0️⃣ Pho Interactive Pipeline Jupyter Widget

In [25]:
import ipywidgets as widgets
from IPython.display import display
from pyphocorehelpers.Filesystem.open_in_system_file_manager import reveal_in_system_file_manager
from pyphoplacecellanalysis.GUI.IPyWidgets.pipeline_ipywidgets import interactive_pipeline_widget, interactive_pipeline_files

_pipeline_jupyter_widget = interactive_pipeline_widget(curr_active_pipeline=curr_active_pipeline)
# display(_pipeline_jupyter_widget)
_pipeline_jupyter_widget

# / 🛑 End Run Section 🛑
-------

# 🎨 2024-02-06 - Other Plotting

In [28]:
from pyphoplacecellanalysis.Pho2D.PyQtPlots.TimeSynchronizedPlotters.TimeSynchronizedPlacefieldsPlotter import TimeSynchronizedPlacefieldsPlotter

_restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')

#  Create a new `SpikeRaster2D` instance using `_display_spike_raster_pyqtplot_2D` and capture its outputs:
curr_active_pipeline.reload_default_display_functions()
curr_active_pipeline.prepare_for_display()

## `LauncherWidget`: GUI

In [29]:
from pyphoplacecellanalysis.General.Pipeline.Stages.Display import DisplayFunctionItem
from pyphocorehelpers.gui.Qt.tree_helpers import find_tree_item_by_text
from pyphoplacecellanalysis.GUI.Qt.MainApplicationWindows.LauncherWidget.LauncherWidget import LauncherWidget

widget = LauncherWidget()
treeWidget = widget.mainTreeWidget # QTreeWidget
widget.build_for_pipeline(curr_active_pipeline=curr_active_pipeline)
widget.show()

setting icon to ":/Render/Icons/Icon/SimplePlot/Laps.png"...
setting icon to ":/Graphics/Icons/graphics/yellow_blue_plot_icon.png"...
setting icon to ":/Render/Icons/Icon/Pseudo2D.png"...
setting icon to ":/Icons/Icons/visualizations/template_1D_debugger.ico"...
setting icon to ":/Graphics/Icons/graphics/directional_track_template_pf1Ds.png"...
setting icon to ":/Icons/Icons/visualizations/rank_order_raster_debugger.ico"...
setting icon to ":/Graphics/Icons/graphics/Spikes.png"...
setting icon to ":/Render/Icons/Icon/Occupancy.png"...
setting icon to ":/Graphics/Icons/graphics/Spikes.png"...
setting icon to ":/Render/Icons/Icon/Occupancy.png"...
setting icon to ":/Render/Icons/Icon/HeatmapUgly.png"...
setting icon to ":/Render/Icons/Icon/Heatmap.png"...
setting icon to ":/Icons/Icons/SpikeRaster2DIcon.ico"...
setting icon to ":/Icons/Icons/SpikeRaster3DIcon.ico"...
setting icon to ":/Icons/Icons/SpikeRaster3D_VedoIcon.ico"...
setting icon to ":/Icons/Icons/InteractivePlaceCellDataExplo

In [ ]:
session_id_str: str = curr_active_pipeline.get_complete_session_identifier_string()
widget.setWindowTitle(f'Spike3D Launcher: {session_id_str}')
treeWidget.root
# curr_active_pipeline.get_session_additional_parameters_context()
# curr_active_pipeline.get_complete_session_context()

## ✅ 2025-09-19 - Clean programmmatic figure outputs 

In [31]:
from pyphocorehelpers.plotting.figure_management import PhoActiveFigureManager2D, capture_new_figures_decorator
fig_man = PhoActiveFigureManager2D(name=f'fig_man') # Initialize a new figure manager
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.DockAreaWrapper import DockAreaWrapper
from pyphoplacecellanalysis.General.Mixins.ExportHelpers import programmatic_render_to_file, programmatic_display_to_PDF, extract_figures_from_display_function_output

fig_man.close_all()

# subset_includelist = ['maze1', 'maze2', 'maze_GLOBAL'] # Day5TwoNovel
# subset_includelist = ['roam', 'sprinkle'] # Day4

subset_includelist = hardcoded_params.decoder_building_session_names
print(f'subset_includelist: {subset_includelist}')

NameError: name 'hardcoded_params' is not defined

In [ ]:
# display_fn_kwargs = dict(subplots=(None, 9))
display_fn_kwargs = dict(subplots=(None, 5))

# _out = dict()
# _out['_display_2d_placefield_result_plot_ratemaps_2D'] = curr_active_pipeline.display(display_function='_display_2d_placefield_result_plot_ratemaps_2D', active_session_configuration_context=IdentifyingContext(format_name='bapun',animal='RatS',session_name='Day5TwoNovel',filter_name='maze1'), **display_fn_kwargs) # _display_2d_placefield_result_plot_ratemaps_2D
# _out['_display_2d_placefield_result_plot_ratemaps_2D'] = curr_active_pipeline.display(display_function='_display_2d_placefield_result_plot_ratemaps_2D', active_session_configuration_context=IdentifyingContext(format_name='bapun',animal='RatS',session_name='Day5TwoNovel',filter_name='maze2'), **display_fn_kwargs) # _display_2d_placefield_result_plot_ratemaps_2D


In [ ]:
_out_list = programmatic_render_to_file(curr_active_pipeline=curr_active_pipeline, curr_display_function_name='_display_2d_placefield_result_plot_ratemaps_2D', subset_includelist=subset_includelist, 
                                        write_vector_format=True, write_png=True, debug_print=True, **display_fn_kwargs)


In [ ]:
_out_list = programmatic_render_to_file(curr_active_pipeline=curr_active_pipeline, curr_display_function_name='_display_2d_placefield_occupancy', subset_includelist=subset_includelist, 
                                        write_vector_format=True, write_png=True, debug_print=True)


## `Spike3DRasterWindowWidget` Cell

In [ ]:
from neuropy.utils.mixins.time_slicing import TimeColumnAliasesProtocol
from neuropy.core.flattened_spiketrains import SpikesAccessor
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.SpikeRasterWidgets.Spike2DRaster import Spike2DRaster
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.helpers import ScrollableRasterViewOwnerMixin
from pyphoplacecellanalysis.GUI.Qt.SpikeRasterWindows.Spike3DRasterWindowWidget import Spike3DRasterWindowWidget
# from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import _setup_spike_raster_window_for_debugging
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.Mixins.Render2DScrollWindowPlot import ScatterItemData # used in `NewSimpleRaster`


active_spikes_df = deepcopy(curr_active_pipeline.sess.spikes_df)

# INLINEING `build_spikes_data_values_from_df`: ______________________________________________________________________ #
# curr_spike_x, curr_spike_y, curr_spike_pens, all_scatterplot_tooltips_kwargs, all_spots, curr_n = cls.build_spikes_data_values_from_df(spikes_df, config_fragile_linear_neuron_IDX_map, is_spike_included=is_spike_included, should_return_data_tooltips_kwargs=should_return_data_tooltips_kwargs, **kwargs)
# All units at once approach:
active_time_variable_name = active_spikes_df.spikes.time_variable_name
print(f'active_time_variable_name: {active_time_variable_name}')
if active_time_variable_name != 't': 
    active_spikes_df = TimeColumnAliasesProtocol.renaming_synonym_columns_if_needed(active_spikes_df, required_columns_synonym_dict={"t":{active_time_variable_name,'t_rel_seconds', 't_seconds'}})
    active_spikes_df = active_spikes_df.drop(columns=[active_time_variable_name], inplace=False) ## drop the old column    
    active_time_variable_name = 't' ## get the new one
    active_spikes_df.spikes.set_time_variable_name('t')
    # default_datapoint_column_names = [active_spikes_df.spikes.time_variable_name, 'aclu', 'fragile_linear_neuron_IDX']
    # active_datapoint_column_names = default_datapoint_column_names
    active_spikes_df
    
# active_spikes_df.spikes.time_variable_name
# active_spikes_df

# Gets the existing SpikeRasterWindow or creates a new one if one doesn't already exist:
# spike_raster_window, (active_2d_plot, active_3d_plot, main_graphics_layout_widget, main_plot_widget, background_static_scroll_plot_widget) = Spike3DRasterWindowWidget.find_or_create_if_needed(curr_active_pipeline, force_create_new=True, allow_replace_hardcoded_main_plots_with_tracks=True)
# spike_raster_window, (active_2d_plot, active_3d_plot, main_graphics_layout_widget, main_plot_widget, background_static_scroll_plot_widget) = Spike3DRasterWindowWidget.find_or_create_if_needed(curr_active_pipeline, force_create_new=False, allow_replace_hardcoded_main_plots_with_tracks=True)
# spike_raster_window, (active_2d_plot, active_3d_plot, *_all_outputs_dict) = Spike3DRasterWindowWidget.find_or_create_if_needed(curr_active_pipeline, force_create_new=False, allow_replace_hardcoded_main_plots_with_tracks=True)
spike_raster_window, (active_2d_plot, active_3d_plot, *_all_outputs_dict) = Spike3DRasterWindowWidget.find_or_create_if_needed(curr_active_pipeline, force_create_new=True, allow_replace_hardcoded_main_plots_with_tracks=True)

active_time_variable_name: t_seconds


	 time variable changed from 't_seconds' to 't'.


	 time variable changed!
no existing SpikeRasterWindow. Creating a new one.
Spike3DRasterBottomPlaybackControlBar.on_start_end_doubleSpinBox_edit_mode_changed(are_controls_editable: False)
missing self.ui.rightSideContainerWidget.ui.neuron_widget_container. returning None.
build_logger(full_logger_string="Spike3D.display.SpikeRasterBase", file_logging_dir: None):
._menuContextAddRenderable is not yet set-up by LocalMenus_AddRenderable.initialize_renderable_context_menu(...). Menu creation will be deferred.
._menuContextAddRenderable is not yet set-up by LocalMenus_AddRenderable.initialize_renderable_context_menu(...). Menu creation will be deferred.
._menuContextAddRenderable is not yet set-up by LocalMenus_AddRenderable.initialize_renderable_context_menu(...). Menu creation will be deferred.
merged_kwargs: {'name': 'spikeRasterOverviewWindowScatterPlotItem', 'pxMode': True, 'symbol': <PyQt5.QtGui.QPainterPath object at 0x000001CD2BBBDC10>, 'size': 5, 'pen': {'color': 'w', 'width': 1},

NotImplementedError: type(curr_sess): <class 'NoneType'>

num_plot_items: 9
plotting item
plotting item
plotting item
plotting item
num_plot_items: 9
plotting item
plotting item
plotting item
plotting item
Spike2DRaster created a new self.ui.main_time_curves_view_widget for TimeCurvesViewMixin plots!
Spike3DRasterWindowWidget._run_delayed_gui_load_code() called!
SpikeRasterBottomFrameControlsMixin_on_window_update(new_start: 10183.064259829973, new_end: 10183.064259829973
Spike2DRaster._run_delayed_gui_load_code() called!
	block_until_render_complete is done. Continuing execution.


	 replacing extant datasource.
num_plot_items: 9
	 replacing extant datasource.
num_plot_items: 9


In [ ]:
# preview_overview_scatter_plot: pg.ScatterPlotItem  = active_2d_plot.plots.preview_overview_scatter_plot # ScatterPlotItem 
# preview_overview_scatter_plot.setDownsampling(auto=True, method='subsample', dsRate=10)
main_graphics_layout_widget: pg.GraphicsLayoutWidget = active_2d_plot.ui.main_graphics_layout_widget
wrapper_layout: pg.QtWidgets.QVBoxLayout = active_2d_plot.ui.wrapper_layout
main_content_splitter = active_2d_plot.ui.main_content_splitter # QSplitter
layout = active_2d_plot.ui.layout
background_static_scroll_window_plot = active_2d_plot.plots.background_static_scroll_window_plot # PlotItem
main_plot_widget = active_2d_plot.plots.main_plot_widget # PlotItem
active_window_container_layout = active_2d_plot.ui.active_window_container_layout # GraphicsLayout, first item of `main_graphics_layout_widget` -- just the active raster window I think, there is a strange black space above it

In [ ]:
print_keys_if_possible('active_2d_plot', active_2d_plot, max_depth=1)

In [ ]:
type(active_2d_plot)

In [ ]:
active_2d_plot.ui


In [ ]:
print_keys_if_possible('active_2d_plot.ui', active_2d_plot.ui, max_depth=2)

In [ ]:
type(spike_raster_window)

In [ ]:
spike_raster_window.ui.wrapper

In [ ]:
active_2d_plot.add_docked_marginal_track(curr_active_pipeline.sess.epochs)
    

In [ ]:
# active_2d_plot.list_all_rendered_intervals()
active_2d_plot.add_laps_intervals(curr_active_pipeline.sess)

active_2d_plot.add_rendered_intervals()



In [ ]:
# curr_active_pipeline.sess.epochs




In [ ]:
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.Mixins.RenderTimeEpochs.Specific2DRenderTimeEpochs import SessionEpochs2DRenderTimeEpochs
from neuropy.core.epoch import ensure_dataframe, ensure_Epoch, Epoch, EpochsAccessor
# SessionEpochs2DRenderTimeEpochs.add_render_time_epochs(curr_sess=curr_active_pipeline.sess.epochs, destination_plot=active_2d_plot)

active_ds = ensure_dataframe(curr_active_pipeline.sess.epochs)
active_ds

_out = active_2d_plot.add_rendered_intervals(active_ds, 'SessionEpochs')

# num_epochs = len(curr_active_pipeline.sess.epochs)

# pen_colors = {'pre': pg.mkColor('purple'), 'roam': pg.mkColor('red'), 'sprinkle': pg.mkColor('red'), 'post': pg.mkColor('purple')}
# brush_colors = {'pre': pg.mkColor('purple'), 'roam': pg.mkColor('red'), 'sprinkle': pg.mkColor('red'), 'post': pg.mkColor('purple')}

# pen_color = list(pen_colors.values())
# brush_color = list(brush_colors.values())
# for a_pen_color in pen_color:
#     a_pen_color.setAlphaF(0.8)

# for a_brush_color in brush_color:
#     a_brush_color.setAlphaF(0.5)


# active_df = cls._update_df_visualization_columns(active_df, y_location, height, pen_color, brush_color, **kwargs)


In [ ]:
active_2d_plot.rendered_epochs


In [ ]:
active_ds = ensure_dataframe(curr_active_pipeline.sess.epochs)
global_epoch_only = ensure_Epoch(active_ds[active_ds['label'] == 'maze_GLOBAL'])
global_epoch_only

curr_active_pipeline.sess
sess.position

## 2025-09-19 - Add Session Paradigm Epochs with a different color for each session

In [ ]:
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.GraphicsWidgets.EpochsEditorItem import EpochsEditor # perform_plot_laps_diagnoser
import matplotlib.pyplot as plt
from pyphocorehelpers.gui.Qt.color_helpers import ColormapHelpers, ColorFormatConverter

def generate_colors(n_epoch):
    cmap = plt.get_cmap('tab20', n_epoch)
    return [plt.matplotlib.colors.rgb2hex(cmap(i)) for i in range(n_epoch)]



sess = curr_active_pipeline.sess # global_session

# pos_df = sess.compute_position_laps() # ensures the laps are computed if they need to be:
position_obj = deepcopy(sess.position)
position_obj.compute_higher_order_derivatives()
pos_df = position_obj.compute_smoothed_position_info(N=20) ## Smooth the velocity curve to apply meaningful logic to it
pos_df = position_obj.to_dataframe()
# Drop rows with missing data in columns: 't', 'velocity_x_smooth' and 2 other columns. This occurs from smoothing
pos_df = pos_df.dropna(subset=['t', 'x_smooth', 'velocity_x_smooth', 'acceleration_x_smooth']).reset_index(drop=True)
# curr_laps_df = sess.laps.to_dataframe()

curr_paradigm_df = ensure_dataframe(sess.paradigm)
curr_paradigm_df = curr_paradigm_df[np.logical_not(np.isin(curr_paradigm_df['label'], ['maze_GLOBAL', 'maze']))] ## exclude the global epoch
n_epochs: int = len(curr_paradigm_df)
# epoch_color_strs: List[str] = generate_colors(n_epochs)
epoch_color_strs: List[str] = [ColorFormatConverter.qColor_to_hexstring(v, include_alpha=False) for v in ColormapHelpers.mpl_to_pg_colormap(mpl_cmap_name='tab20', resolution=n_epochs).getColors(mode='qcolor')]
curr_paradigm_df['lap_color'] = "#10FF44"
curr_paradigm_df['lap_color'] = epoch_color_strs
curr_paradigm_df['lap_accent_color'] = '#FFFFFF'
curr_paradigm_df
## Create a new window:
custom_epoch_label_kwargs = dict(epoch_label_position=0.05, epoch_label_rotateAxis=(0,0), epoch_label_anchor=(0.0, 1.0))
epochs_editor = EpochsEditor.init_laps_diagnoser(pos_df, curr_paradigm_df, include_velocity=False, include_accel=False, span=(0.05, 0.95), movable=False, **custom_epoch_label_kwargs)

In [ ]:
epochs_editor.plots
# epochs_editor.rebuild_epoch_regions()



In [ ]:
active_2d_plot.plots.main_plot_widget

main_plot_widget = active_2d_plot.plots.main_plot_widget # PlotItem
main_plot_widget.setMinimumHeight(20.0)


In [ ]:
# active_window_container_layout
# main_graphics_layout_widget.ci # GraphicsLayout
main_graphics_layout_widget.ci.childItems()
# main_graphics_layout_widget.setHidden(True) ## hides too much
main_graphics_layout_widget.setHidden(False)

# main_graphics_layout_widget

active_window_container_layout.setBorder(pg.mkPen('yellow', width=4.5))

In [ ]:
# active_window_container_layout.allChildItems()
active_window_container_layout.setPreferredHeight(200.0)
active_window_container_layout.setMaximumHeight(800.0)
active_window_container_layout.setSpacing(0)

In [ ]:
# Set stretch factors to control priority
main_graphics_layout_widget.ci.layout.setRowStretchFactor(0, 400)  # Plot1: lowest priority
main_graphics_layout_widget.ci.layout.setRowStretchFactor(1, 2)  # Plot2: mid priority
main_graphics_layout_widget.ci.layout.setRowStretchFactor(2, 2)  # Plot3: highest priority


In [ ]:
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.ParameterTreeWidget import create_parameter_tree_widget
# win, param_tree = create_pipeline_filter_parameter_tree()
win, param_tree = create_parameter_tree_widget(curr_active_pipeline.get_all_parameters())
win.show()

In [37]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.DirectionalPlacefieldGlobalComputationFunctions import DirectionalDecodersContinuouslyDecodedResult
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.SpikeRasterWidgets.Spike2DRaster import SynchronizedPlotMode
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.DecoderPredictionError import plot_1D_most_likely_position_comparsions
from pyphoplacecellanalysis.General.Model.Configs.LongShortDisplayConfig import DecoderIdentityColors
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import _perform_plot_multi_decoder_meas_pred_position_track
from pyphoplacecellanalysis.Analysis.Decoder.reconstruction import DecodedFilterEpochsResult

## Build the new dock track:
dock_identifier: str = 'Continuous Decoding Performance'
ts_widget, fig, ax_list, dDisplayItem = active_2d_plot.add_new_matplotlib_render_plot_widget(name=dock_identifier)
## Get the needed data:
directional_decoders_decode_result: DirectionalDecodersContinuouslyDecodedResult = curr_active_pipeline.global_computation_results.computed_data['DirectionalDecodersDecoded']
all_directional_pf1D_Decoder_dict: Dict[str, BasePositionDecoder] = directional_decoders_decode_result.pf1D_Decoder_dict
continuously_decoded_result_cache_dict = directional_decoders_decode_result.continuously_decoded_result_cache_dict
previously_decoded_keys: List[float] = list(continuously_decoded_result_cache_dict.keys()) # [0.03333]
print(F'previously_decoded time_bin_sizes: {previously_decoded_keys}')

time_bin_size: float = directional_decoders_decode_result.most_recent_decoding_time_bin_size
print(f'time_bin_size: {time_bin_size}')
continuously_decoded_dict: Dict[str, DecodedFilterEpochsResult] = directional_decoders_decode_result.most_recent_continuously_decoded_dict
all_directional_continuously_decoded_dict: Dict[types.DecoderName, DecodedFilterEpochsResult] = {k:v for k, v in (continuously_decoded_dict or {}).items() if k in TrackTemplates.get_decoder_names()} ## what is plotted in the `f'{a_decoder_name}_ContinuousDecode'` rows by `AddNewDirectionalDecodedEpochs_MatplotlibPlotCommand`
## OUT: all_directional_continuously_decoded_dict
## Draw the position meas/decoded on the plot widget
## INPUT: fig, ax_list, all_directional_continuously_decoded_dict, track_templates

_out_artists =  _perform_plot_multi_decoder_meas_pred_position_track(curr_active_pipeline, fig, ax_list, desired_time_bin_size=0.058, enable_flat_line_drawing=True)

## sync up the widgets
active_2d_plot.sync_matplotlib_render_plot_widget(dock_identifier, sync_mode=SynchronizedPlotMode.TO_WINDOW)

KeyError: 'DirectionalDecodersDecoded'

In [ ]:
pos_df['truth_decoder_name'] = pos_df['truth_decoder_name'].fillna('')
pos_df

In [ ]:
decoder_color_dict: Dict[types.DecoderName, str] = DecoderIdentityColors.build_decoder_color_dict()

decoded_pos_line_kwargs = dict(lw=1.0, color='gray', alpha=0.8, marker='+', markersize=6, animated=False)
inactive_decoded_pos_line_kwargs = dict(lw=0.3, alpha=0.2, marker='.', markersize=2, animated=False)
active_decoded_pos_line_kwargs = dict(lw=1.0, alpha=0.8, marker='+', markersize=6, animated=False)


_out_data = {}
_out_data_plot_kwargs = {}
# curr_active_pipeline.global_computation_results.t
for a_decoder_name, a_decoder in track_templates.get_decoders_dict().items():
    a_continuously_decoded_result = all_directional_continuously_decoded_dict[a_decoder_name]
    a_decoder_color = decoder_color_dict[a_decoder_name]
    
    assert len(a_continuously_decoded_result.p_x_given_n_list) == 1
    p_x_given_n = a_continuously_decoded_result.p_x_given_n_list[0]
    # p_x_given_n = a_continuously_decoded_result.p_x_given_n_list[0]['p_x_given_n']
    time_bin_containers = a_continuously_decoded_result.time_bin_containers[0]
    time_window_centers = time_bin_containers.centers
    # p_x_given_n.shape # (62, 4, 209389)
    a_marginal_x = a_continuously_decoded_result.marginal_x_list[0]
    # active_time_window_variable = a_decoder.active_time_window_centers
    active_time_window_variable = time_window_centers
    active_most_likely_positions_x = a_marginal_x['most_likely_positions_1D'] # a_decoder.most_likely_positions[:,0].T
    _out_data[a_decoder_name] = pd.DataFrame({'t': time_window_centers, 'x': active_most_likely_positions_x, 'binned_time': np.arange(len(time_window_centers))})
    _out_data[a_decoder_name] = _out_data[a_decoder_name].position.adding_lap_info(laps_df=laps_df, inplace=False)
    _out_data[a_decoder_name] = _out_data[a_decoder_name].time_point_event.adding_true_decoder_identifier(t_start=t_start, t_delta=t_delta, t_end=t_end) ## ensures ['maze_id', 'is_LR_dir']
    _out_data[a_decoder_name]['is_active_decoder_time'] = (_out_data[a_decoder_name]['truth_decoder_name'].fillna('', inplace=False) == a_decoder_name)

    # is_active_decoder_time = (_out_data[a_decoder_name]['truth_decoder_name'] == a_decoder_name)
    active_decoder_time_points = _out_data[a_decoder_name][_out_data[a_decoder_name]['truth_decoder_name'] == a_decoder_name]['t'].to_numpy()
    active_decoder_most_likely_positions_x = _out_data[a_decoder_name][_out_data[a_decoder_name]['truth_decoder_name'] == a_decoder_name]['x'].to_numpy()
    active_decoder_inactive_time_points = _out_data[a_decoder_name][_out_data[a_decoder_name]['truth_decoder_name'] != a_decoder_name]['t'].to_numpy()
    active_decoder_inactive_most_likely_positions_x = _out_data[a_decoder_name][_out_data[a_decoder_name]['truth_decoder_name'] != a_decoder_name]['x'].to_numpy()
    ## could fill y with np.nan instead of getting shorter?
    _out_data_plot_kwargs[a_decoder_name] = (dict(x=active_decoder_time_points, y=active_decoder_most_likely_positions_x, color=a_decoder_color, **active_decoded_pos_line_kwargs), dict(x=active_decoder_inactive_time_points, y=active_decoder_inactive_most_likely_positions_x, color=a_decoder_color, **inactive_decoded_pos_line_kwargs))

_out_data_plot_kwargs

In [ ]:
# _out_data[a_decoder_name] = _out_data[a_decoder_name].position.adding_lap_info(laps_df=laps_df, inplace=False)
# _out_data[a_decoder_name] = _out_data[a_decoder_name].time_point_event.adding_true_decoder_identifier(t_start=t_start, t_delta=t_delta, t_end=t_end) ## ensures ['maze_id', 'is_LR_dir']

# is_active_decoder_time = (_out_data[a_decoder_name]['truth_decoder_name'] == a_decoder_name)
active_decoder_time_points = _out_data[a_decoder_name][_out_data[a_decoder_name]['truth_decoder_name'] == a_decoder_name]['t'].to_numpy()
active_decoder_most_likely_positions_x = _out_data[a_decoder_name][_out_data[a_decoder_name]['truth_decoder_name'] == a_decoder_name]['x'].to_numpy()
active_decoder_inactive_time_points = _out_data[a_decoder_name][_out_data[a_decoder_name]['truth_decoder_name'] != a_decoder_name]['t'].to_numpy()
active_decoder_inactive_most_likely_positions_x = _out_data[a_decoder_name][_out_data[a_decoder_name]['truth_decoder_name'] != a_decoder_name]['x'].to_numpy()

_out_data[a_decoder_name] = ((active_decoder_time_points, active_decoder_most_likely_positions_x), (active_decoder_inactive_time_points, active_decoder_inactive_most_likely_positions_x))


In [ ]:
partitioned_dfs = partition_df_dict(pos_df, partitionColumn='truth_decoder_name')

a_decoder_name: str = 'short_LR'
a_binned_time_grouped_df = partitioned_dfs[a_decoder_name].groupby('binned_time', axis='index', dropna=True)
a_binned_time_grouped_df = a_binned_time_grouped_df.median().dropna(axis='index', subset=['x']) ## without the `.dropna(axis='index', subset=['x'])` part it gets an exhaustive df for all possible values of 'binned_time', even those not listed

a_matching_binned_times = a_binned_time_grouped_df.reset_index(drop=False)['binned_time']
a_matching_binned_times

In [ ]:
## split into two dfs for each decoder -- the supported and the unsupported
partition

PandasHelpers.safe_pandas_get_group

In [ ]:
pos_df.dropna(axis='index', subset=['lap', 'truth_decoder_name'], inplace=False)

In [ ]:
laps_df: pd.DataFrame = global_laps_obj.to_dataframe()

In [ ]:
from neuropy.core.epoch import find_epochs_overlapping_other_epochs

## INPUTS: global_laps
_out_split_pseudo2D_posteriors_dict = {}
_out_split_pseudo2D_out_dict = {}
pre_filtered_col_names = ['pre_filtered_most_likely_position_indicies', 'pre_filtered_most_likely_position'] # 'pre_filtered_time_bin_containers', 'pre_filtered_p_x_given_n', 
post_filtered_col_names = [a_col_name.removeprefix('pre_filtered_') for a_col_name in pre_filtered_col_names] # ['time_bin_containers', 'most_likely_position_indicies', 'most_likely_position']
print(post_filtered_col_names)
for a_time_bin_size, pseudo2D_decoder_continuously_decoded_result in continuously_decoded_pseudo2D_decoder_dict.items():
    print(f'a_time_bin_size: {a_time_bin_size}')
    _out_split_pseudo2D_out_dict[a_time_bin_size] = {'pre_filtered_p_x_given_n': None, 'pre_filtered_time_bin_containers': None, 'pre_filtered_most_likely_position_indicies': None, 'pre_filtered_most_likely_position': None, 
                                                     'is_timebin_included': None, 'p_x_given_n': None} # , 'time_window_centers': None
    # pseudo2D_decoder_continuously_decoded_result: DecodedFilterEpochsResult = continuously_decoded_dict.get('pseudo2D', None)
    assert len(pseudo2D_decoder_continuously_decoded_result.p_x_given_n_list) == 1
    p_x_given_n = pseudo2D_decoder_continuously_decoded_result.p_x_given_n_list[0]
    # p_x_given_n = pseudo2D_decoder_continuously_decoded_result.p_x_given_n_list[0]['p_x_given_n']
    time_bin_containers = pseudo2D_decoder_continuously_decoded_result.time_bin_containers[0]
    # time_window_centers = time_bin_containers.centers
    _out_split_pseudo2D_out_dict[a_time_bin_size]['pre_filtered_most_likely_position_indicies'] = deepcopy(pseudo2D_decoder_continuously_decoded_result.most_likely_position_indicies_list[0])
    _out_split_pseudo2D_out_dict[a_time_bin_size]['pre_filtered_most_likely_position'] = deepcopy(pseudo2D_decoder_continuously_decoded_result.most_likely_positions_list[0])
    ## INPUTS: time_bin_containers, global_laps
    left_edges = deepcopy(time_bin_containers.left_edges)
    right_edges = deepcopy(time_bin_containers.right_edges)
    continuous_time_binned_computation_epochs_df: pd.DataFrame = pd.DataFrame({'start': left_edges, 'stop': right_edges, 'label': np.arange(len(left_edges))})
    is_timebin_included: NDArray = find_epochs_overlapping_other_epochs(epochs_df=continuous_time_binned_computation_epochs_df, epochs_df_required_to_overlap=deepcopy(global_laps))
    _out_split_pseudo2D_out_dict[a_time_bin_size]['pre_filtered_p_x_given_n'] = p_x_given_n
    _out_split_pseudo2D_out_dict[a_time_bin_size]['pre_filtered_time_bin_containers'] = time_bin_containers
    _out_split_pseudo2D_out_dict[a_time_bin_size]['is_timebin_included'] = is_timebin_included
    # continuous_time_binned_computation_epochs_df['is_in_laps'] = is_timebin_included
    ## filter by whether it's included or not:
    p_x_given_n = p_x_given_n[:, :, is_timebin_included]
    # time_window_centers = 
    _out_split_pseudo2D_out_dict[a_time_bin_size]['p_x_given_n'] = p_x_given_n
    # _out_split_pseudo2D_out_dict[a_time_bin_size]['time_window_centers'] = time_window_centers[is_timebin_included]
    # p_x_given_n.shape # (62, 4, 209389)

    ## Split across the 2nd axis to make 1D posteriors that can be displayed in separate dock rows:
    assert p_x_given_n.shape[1] == 4, f"expected the 4 pseudo-y bins for the decoder in p_x_given_n.shape[1]. but found p_x_given_n.shape: {p_x_given_n.shape}"
    # split_pseudo2D_posteriors_dict = {k:np.squeeze(p_x_given_n[:, i, :]) for i, k in enumerate(('long_LR', 'long_RL', 'short_LR', 'short_RL'))}
    _out_split_pseudo2D_posteriors_dict[a_time_bin_size] = deepcopy(p_x_given_n)
    
    # for a_col_name in pre_filtered_col_names:
    #     filtered_col_name = a_col_name.removeprefix('pre_filtered_')
    #     print(f'a_col_name: {a_col_name}, filtered_col_name: {filtered_col_name}, shape: {np.shape(_out_split_pseudo2D_out_dict[a_time_bin_size][a_col_name])}')
    #     _out_split_pseudo2D_out_dict[a_time_bin_size][filtered_col_name] = _out_split_pseudo2D_out_dict[a_time_bin_size][a_col_name][is_timebin_included, :]
        
    _out_split_pseudo2D_out_dict[a_time_bin_size]['most_likely_position_indicies'] = _out_split_pseudo2D_out_dict[a_time_bin_size]['pre_filtered_most_likely_position_indicies'][:, is_timebin_included]
    _out_split_pseudo2D_out_dict[a_time_bin_size]['most_likely_position'] = _out_split_pseudo2D_out_dict[a_time_bin_size]['pre_filtered_most_likely_position'][is_timebin_included, :]
    

p_x_given_n.shape # (n_position_bins, n_decoding_models, n_time_bins) - (57, 4, 29951)

## OUTPUTS: _out_split_pseudo2D_posteriors_dict, _out_split_pseudo2D_out_dict

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.DecoderPredictionError import plot_most_likely_position_comparsions

# fig, axs = plot_most_likely_position_comparsions(pho_custom_decoder, axs=ax, sess.position.to_dataframe())
fig, axs = plot_most_likely_position_comparsions(computation_result.computed_data['pf2D_Decoder'], computation_result.sess.position.to_dataframe(), **overriding_dict_with(lhs_dict={'show_posterior':True, 'show_one_step_most_likely_positions_plots':True}, **kwargs))


# 🖼️⚓❎ Time Synchronized Plotting with position

In [ ]:
from pyphoplacecellanalysis.Pho2D.PyQtPlots.TimeSynchronizedPlotters.Mixins.AnimalTrajectoryPlottingMixin import AnimalTrajectoryPlottingMixin
from pyphoplacecellanalysis.Pho2D.PyQtPlots.TimeSynchronizedPlotters.TimeSynchronizedPositionDecoderPlotter import TimeSynchronizedPositionDecoderPlotter
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.ContainerBased.PhoContainerTool import GenericPyQtGraphContainer
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import build_combined_time_synchronized_Bapun_decoders_window
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.SpikeRasterWidgets.Spike2DRaster import Spike2DRaster, SynchronizedPlotMode
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.DockAreaWrapper import PhoDockAreaContainingWindow
from pyphoplacecellanalysis.GUI.PyQtPlot.DockingWidgets.SpecificDockWidgetManipulatingMixin import SpecificDockWidgetManipulatingMixin

hardcoded_params: HardcodedProcessingParameters = BapunDataSessionFormatRegisteredClass._get_session_specific_parameters(session_context=curr_active_pipeline.get_session_context())
# hardcoded_params.decoder_building_session_names
# hardcoded_params.non_global_activity_session_names

# pg.setConfigOptions(useOpenGL=True)  # do this BEFORE creating plots/widgets

## Uses the `global_computation_results.computed_data['DirectionalDecodersDecoded']`
# directional_decoders_decode_result: DirectionalDecodersContinuouslyDecodedResult = curr_active_pipeline.global_computation_results.computed_data['DirectionalDecodersDecoded']

# _out_container: GenericPyQtGraphContainer = build_combined_time_synchronized_Bapun_decoders_window(curr_active_pipeline, included_filter_names=hardcoded_params.non_global_activity_session_names, fixed_window_duration = 3.0)
_out_container_new: GenericPyQtGraphContainer = build_combined_time_synchronized_Bapun_decoders_window(curr_active_pipeline, included_filter_names=hardcoded_params.non_global_activity_session_names, fixed_window_duration = 3.0,
    directional_decoders_decode_result=directional_decoders_decode_result,
	controlling_widget=active_2d_plot, create_new_controlling_widget=False,
)

active_2d_plot: Spike2DRaster = _out_container_new.ui.controlling_widget
sync_plotters: Dict[str, TimeSynchronizedPositionDecoderPlotter] = _out_container_new.ui.sync_plotters
win: PhoDockAreaContainingWindow = _out_container_new.ui.root_dockAreaWindow
# a_sync_plotter: TimeSynchronizedPositionDecoderPlotter = sync_plotters['roam']
# a_sync_plotter.curr_position

# ## Disable debug print to speed up animation
# for a_plotter_name, a_plotter in sync_plotters.items():
#     a_plotter.params.debug_print = False


## INPUTS: _out_container, active_2d_plot, _out_container, sync_plotters, 


In [ ]:
_out_pbe_tracks, _out_pbe_overview_tracks = _out_container_new.add_pbes_full_result_marginals(pbes_full_result=pbes_full_result)

# grouped_dock_items_dict = _out_container_new.build_overview_and_windowed_dockgroups() ## Not quite ready

## Reorder the dock items

In [ ]:
list(active_2d_plot.dock_manager_widget.dynamic_display_dict.keys())
# original_identifier_order = ['interval_overview', 'intervals', 'rasters[raster_overview]', 'rasters[raster_window]', 'global context', 'global context (overview)', 'pbe[0.06]', 'pbe[0.06] (Overview)']
desired_identifier_order = ['interval_overview', 'rasters[raster_overview]', 'global context (overview)', 'pbe[0.06] (Overview)', 'rasters[raster_window]', 'intervals', 'global context', 'pbe[0.06]'] ## #TODO 2025-09-21 15:41: - [ ] Enforce this order programmatically?!


In [ ]:
active_2d_plot.dock_manager_widget

In [ ]:

## INPUTS: active_2d_plot
grouped_dock_items_dict = active_2d_plot.ui.dynamic_docked_widget_container.get_dockGroup_dock_dict()
nested_dock_items = {}
nested_dynamic_docked_widget_container_widgets = {}
for dock_group_name, flat_group_dockitems_list in grouped_dock_items_dict.items():
    dDisplayItem, nested_dynamic_docked_widget_container = active_2d_plot.ui.dynamic_docked_widget_container.build_wrapping_nested_dock_area(flat_group_dockitems_list, dock_group_name=dock_group_name)
    nested_dock_items[dock_group_name] = dDisplayItem
    nested_dynamic_docked_widget_container_widgets[dock_group_name] = nested_dynamic_docked_widget_container

## OUTPUTS: nested_dock_items, nested_dynamic_docked_widget_container_widgets

In [ ]:
grouped_dock_items_dict = active_2d_plot.ui.dynamic_docked_widget_container.get_dockGroup_dock_dict()
grouped_dock_items_dict

In [ ]:



grouped_dock_items_dict = build_overview_and_windowed_dockgroups(active_2d_plot)
grouped_dock_items_dict


In [ ]:
_out = active_2d_plot.dock_manager_widget.layout_dockGroups()


### 📈🔃❎ Exporting as video

In [ ]:
from PyQt5 import QtWidgets, QtGui, QtCore
import pyphoplacecellanalysis.External.pyqtgraph as pg
from pyqtgraph.exporters import ImageExporter
from PIL import Image
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.GraphicsWidgets.CustomGraphicsLayoutWidget import CustomGraphicsLayoutWidget

## "playback" refers to output video:
desired_playback_duration: float = 8 * 60.0 # 8m

# "session" refers to the actual recording session:
desired_session_time_range_duration: float = (16.0 * 60.0) # 1m
# session_start_t: float = 23170.37047485091 #20740.63578640229 # 11631.186907154472 # 11451.186907154472 # 11391.186907154472 # 7665.232126354053 # active_2d_plot.total_data_start_time + 4.0 * 60.0 # 4 minutes into start of recording ## Day 5
# session_start_t: float = 11023.018433333335 # 10843.018433333334 # active_2d_plot.total_data_start_time + 4.0 * 60.0 # 4 minutes into start of recording
# session_start_t: float = 23170.37047485091 #20740.63578640229 # 11631.186907154472 # 11451.186907154472 # 11391.186907154472 # active_2d_plot.total_data_start_time + 4.0 * 60.0 # 4 minutes into start of recording ## day 4

session_start_t: float = active_2d_plot.total_data_start_time + 4.0 * 60.0 # 4 minutes into start of recording ## day 4

desired_session_time_range: Tuple[float, float] = (session_start_t, (session_start_t + desired_session_time_range_duration))

playback_speed_factor: float = (desired_playback_duration / desired_session_time_range_duration)

print(f'playback_speed_factor: {playback_speed_factor}')
time_window_duration: float = active_2d_plot.active_window_duration
print(f'time_window_duration: {time_window_duration}')

## INPUTS: _out_container, active_2d_plot, _out_container, sync_plotters, 
desired_framerate: float = 2.0
desired_frame_duration_sec: float = 1.0/desired_framerate
print(f'desired_frame_duration_sec: {desired_frame_duration_sec}')

# ## All Frames from entire recording (too long)
# total_duration: float = active_2d_plot.total_data_duration
# desired_num_total_frames: int = int(np.ceil((total_duration * desired_framerate)))
# frame_start_indicies = np.linspace(active_2d_plot.total_data_start_time,  active_2d_plot.total_data_end_time, num=desired_num_total_frames)

## Plot only for the range of interest:
desired_num_total_frames: int = int(np.ceil((desired_session_time_range_duration * desired_framerate)))
frame_start_indicies = np.linspace(desired_session_time_range[0], desired_session_time_range[1], num=desired_num_total_frames)
frame_end_indices = frame_start_indicies + desired_frame_duration_sec

print(f'desired_num_total_frames: {desired_num_total_frames}')


In [ ]:
# ## Disable debug print to speed up animation
for a_plotter_name, a_plotter in sync_plotters.items():
    a_plotter.params.debug_print = False

In [ ]:
active_2d_plot.active_window_start_time

In [ ]:
# next_end_timestamp = next_start_timestamp + self.animation_active_time_window.window_duration

def _frame_update(frame_start_t, frame_end_t):
    active_2d_plot.update_scroll_window_region(frame_start_t, frame_end_t, block_signals=True)
    active_2d_plot.window_scrolled.emit(frame_start_t, frame_end_t)
    QtWidgets.QApplication.processEvents()
    win.repaint()


_frame_update(desired_session_time_range[0], (desired_session_time_range[0]+time_window_duration))


timestep_delta_sec: float = 0.5 # half second step


In [ ]:
## Step one frame:
frame_start_t: float = active_2d_plot.active_window_start_time + timestep_delta_sec ## current time plus delta
_frame_update(frame_start_t, (frame_start_t + time_window_duration))

In [ ]:
for i, (frame_start_t, frame_end_t) in enumerate(zip(frame_start_indicies, frame_end_indices)):
    print(f'frame[{i}]: ({frame_start_t}, {frame_end_t}):')
    # active_2d_plot.on_window_changed(frame_start_t, frame_end_t)
    # active_2d_plot.update_scroll_window_region(frame_start_t, frame_end_t, block_signals=True)
    # active_2d_plot.window_scrolled.emit(frame_start_t, frame_end_t)
    # pg.SignalProxy(driver.window_scrolled, delay=0.2, rateLimit=60, slot=drivable.on_window_changed_rate_limited)
    # QtWidgets.QApplication.processEvents()
    # win.repaint()
    # _frame_update(frame_start_t, frame_end_t)
    _frame_update(frame_start_t, (frame_start_t + time_window_duration))


##### NOT WORKING YET: Export both to a multi-page PDF or video


In [ ]:
from pyphoplacecellanalysis.General.Mixins.ExportHelpers import FigureToImageHelpers

all_tracks_export_path = curr_active_pipeline.get_output_path().joinpath('2025-09-19_910pm_time_synch_dual_plotters.pdf')

## INPUTS: 
_out_container: GenericPyQtGraphContainer
active_2d_plot: Spike2DRaster = _out_container.ui.controlling_widget
sync_plotters = _out_container.ui.sync_plotters

saved_output_pdf_path = FigureToImageHelpers.export_wrapped_tracks_to_paged_df(active_2d_plot, output_pdf_path=all_tracks_export_path)
print(f'saved_output_pdf_path: {saved_output_pdf_path}')


In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.DecoderPredictionError import _temp_debug_two_step_plots_animated_pyqtgraph

# Simple plot type 1:
# plotted_variable_name = kwargs.get('variable_name', 'p_x_given_n') # Tries to get the user-provided variable name, otherwise defaults to 'p_x_given_n'
plotted_variable_name = 'p_x_given_n' # Tries to get the user-provided variable name, otherwise defaults to 'p_x_given_n'
_temp_debug_two_step_plots_animated_pyqtgraph(active_one_step_decoder, active_two_step_decoder, active_computed_data.extended_stats.time_binned_position_df, variable_name=plotted_variable_name) # Works

### OLD EXPLORE ONLY

In [ ]:
a_plotter.params.trajectory_path_marker_max_fill_opacity

In [ ]:
a_plotter.params.recent_position_trajectory_max_seconds_ago

In [ ]:
a_plotter.curr_recent_trajectory

In [ ]:
# a_plotter.ui.im
import pyphoplacecellanalysis.External.pyqtgraph as pg

# from PyQt5 import QtGui
# import pyphoplacecellanalysis.External.pyqtgraph.Qt as pg

from PyQt5.QtGui import QPainter


traj_curve_item: pg.PlotDataItem = a_plotter.ui.trajectory_curve
# traj_curve_item.show()



traj_curve_item.setAlpha(1.0, auto=False)
traj_curve_item.setSymbolBrush((50, 50, 50, 25))


# from QtGui.QPainter import 
# pg.QtGui.QPainter

# from pg import QtGui
# from PyQt5.QtGui.QPainter import CompositionMode

# from pg.QtGui.QPainter import CompositionMode
# QPainter.CompositionMode.CompositionMode_Overlay


In [ ]:


# a_plotter.ui.imv.setCompositionMode(QPainter.CompositionMode_Multiply)
# a_plotter.ui.imv.setCompositionMode(QPainter.CompositionMode_Overlay)
a_plotter.ui.imv.setCompositionMode(QPainter.CompositionMode_Plus)



In [ ]:
a_plotter.active_one_step_decoder.time_window_centers


In [ ]:
# a_plotter.ui.trajectory_curve.
a_plotter.params.recent_position_trajectory_path_shadow_pen
a_plotter.params.recent_position_trajectory_path_pen



In [ ]:
a_plotter.params.recent_position_trajectory_symbol_pen

pg.mkPen(a_plotter.params.recent_position_trajectory_symbol_pen.color(), 

In [ ]:
a_plotter.params.recent_position_trajectory_symbol_pen.color().setAlphaF(0.1)

pg.mkBrush(

In [ ]:
a_plotter.params

In [ ]:
a_plotter.ui.trajectory_curve.setPen(a_plotter.params.recent_position_trajectory_symbol_pen)

In [ ]:
sync_plotters

### Build Marginals over track context and plot them on the timeline

In [ ]:
from pyphoplacecellanalysis.Analysis.Decoder.reconstruction import DecodedFilterEpochsResult, SingleEpochDecodedResult
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import _add_context_marginal_to_timeline, _add_context_decoded_epoch_marginals_to_timeline


In [ ]:

# decoded_epochs_track_name: str = f'{epochs_name}[{decoding_time_bin_size}]'


_out = _add_context_marginal_to_timeline(active_2d_plot, a_filter_epochs_decoded_result=all_context_filter_epochs_decoder_result, name='global context')
_out_pbe_tracks = _add_context_decoded_epoch_marginals_to_timeline(active_2d_plot=active_2d_plot, decoded_epochs_result=pbe_decoder_result, name=f"pbe[{decoding_time_bin_size}]")


In [ ]:
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.SpikeRasterWidgets.Spike2DRaster import SynchronizedPlotMode

_out_new = _add_context_marginal_to_timeline(active_2d_plot, a_filter_epochs_decoded_result=all_context_filter_epochs_decoder_result, name='global context (overview)')
_out_new

identifier_name, widget, matplotlib_fig, matplotlib_fig_axes, dock_item = _out_new

## 🟢🎨 Add the Session Paradigm/PBEs Epoch Intervals, then color and position them on the timeline


In [ ]:
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.Mixins.RenderTimeEpochs.Specific2DRenderTimeEpochs import SessionEpochs2DRenderTimeEpochs
from neuropy.core.epoch import ensure_dataframe, ensure_Epoch, Epoch, EpochsAccessor
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.Mixins.RenderTimeEpochs.EpochRenderingMixin import EpochRenderingMixin
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.Mixins.RenderTimeEpochs.Specific2DRenderTimeEpochs import General2DRenderTimeEpochs, Ripples_2DRenderTimeEpochs


    # _out_pbe_tracks, _out_pbe_overview_tracks = add_pbes_full_result_marginals(active_2d_plot, pbes_full_result)

_out_pbe_tracks, _out_pbe_overview_tracks = _out_container.add_pbes_full_result_marginals(pbes_full_result=pbes_full_result)

#### Overflow unused

In [ ]:
### NOT NEEDED:
# active_df = cls._update_df_visualization_columns(active_df, y_location, height, pen_color, brush_color, **kwargs)

rendered_interval_keys = ['_', 'SessionEpochs', '_', 'PBEs'] # '_' indicates a vertical spacer

# desired_interval_height_ratios = [2.0, 2.0, 1.0, 0.1, 1.0, 1.0, 1.0] # ratio of heights to each interval
desired_interval_height_ratios = [1.0, 2.0, 0.1, 0.5] # ratio of heights to each interval

required_vertical_offsets, required_interval_heights = EpochRenderingMixin.build_stacked_epoch_layout(desired_interval_height_ratios, epoch_render_stack_height=20.0, interval_stack_location='below')
stacked_epoch_layout_dict = {interval_key:dict(y_location=y_location, height=height) for interval_key, y_location, height in zip(rendered_interval_keys, required_vertical_offsets, required_interval_heights)}

active_2d_plot.update_rendered_intervals_visualization_properties({'SessionEpochs': dict()})

a_ds = active_2d_plot.interval_datasources.SessionEpochs
a_ds.update_visualization_properties(_updated_custom_interval_dataframe_visualization_columns_general_epoch)                                         

In [ ]:
active_2d_plot.clear_all_rendered_intervals()

In [ ]:
a_ds = active_2d_plot.interval_datasources.SessionEpochs
a_ds.update_visualization_properties(_updated_custom_interval_dataframe_visualization_columns_general_epoch)



In [ ]:

# SessionEpochs2DRenderTimeEpochs.add_render_time_epochs(curr_sess=curr_active_pipeline.sess.epochs, destination_plot=active_2d_plot)

active_pbe_ds = ensure_dataframe(curr_active_pipeline.sess.pbe)
_out_pbes = active_2d_plot.add_rendered_intervals(active_pbe_ds, 'PBEs')
_out_pbes


In [ ]:
active_2d_plot.interval_datasources.PBEs.update_visualization_properties(lambda active_df, **kwargs: General2DRenderTimeEpochs._update_df_visualization_columns(active_df, y_location=-12.0, height=1.5, pen_color=inline_mkColor('cyan', 0.8), brush_color=inline_mkColor('cyan', 0.5), **kwargs)) ## Fully inline

In [ ]:

# _out

active_2d_plot.get_all_rendered_intervals_dict()

# from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.Mixins.RenderTimeEpochs.EpochRenderingMixin import EpochRenderingMixin

# rendered_interval_keys = ['_', 'SessionEpochs', 'Laps', '_', 'PBEs', 'Ripples', 'Replays'] # '_' indicates a vertical spacer

# desired_interval_height_ratios = [2.0, 2.0, 1.0, 0.1, 1.0, 1.0, 1.0] # ratio of heights to each interval
# required_vertical_offsets, required_interval_heights = EpochRenderingMixin.build_stacked_epoch_layout(desired_interval_height_ratios, epoch_render_stack_height=20.0, interval_stack_location='below')
# stacked_epoch_layout_dict = {interval_key:dict(y_location=y_location, height=height) for interval_key, y_location, height in zip(rendered_interval_keys, required_vertical_offsets, required_interval_heights)}

# num_epochs = len(curr_active_pipeline.sess.epochs)

# pen_colors = {'pre': pg.mkColor('purple'), 'roam': pg.mkColor('red'), 'sprinkle': pg.mkColor('red'), 'post': pg.mkColor('purple')}
# brush_colors = {'pre': pg.mkColor('purple'), 'roam': pg.mkColor('red'), 'sprinkle': pg.mkColor('red'), 'post': pg.mkColor('purple')}

# pen_color = list(pen_colors.values())
# brush_color = list(brush_colors.values())
# for a_pen_color in pen_color:
#     a_pen_color.setAlphaF(0.8)

# for a_brush_color in brush_color:
#     a_brush_color.setAlphaF(0.5)


# active_df = cls._update_df_visualization_columns(active_df, y_location, height, pen_color, brush_color, **kwargs)


In [ ]:
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.Mixins.RenderTimeEpochs.Specific2DRenderTimeEpochs import General2DRenderTimeEpochs, inline_mkColor

active_2d_plot.get_all_rendered_intervals_dict()
active_2d_plot.interval_datasources.SessionEpochs.update_visualization_properties(lambda active_df, **kwargs: General2DRenderTimeEpochs._update_df_visualization_columns(active_df, y_location=-12.0, height=1.5, pen_color=inline_mkColor('cyan', 0.8), brush_color=inline_mkColor('cyan', 0.5), **kwargs)) ## Fully inline

# active_2d_plot.add_rendered_intervals(
    

#### (OTHER) Independent Epochs with `EpochsEditor` which looks much nicer

In [ ]:
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.GraphicsWidgets.EpochsEditorItem import EpochsEditor # perform_plot_laps_diagnoser
import matplotlib.pyplot as plt
from pyphocorehelpers.gui.Qt.color_helpers import ColormapHelpers, ColorFormatConverter

def generate_colors(n_epoch):
    cmap = plt.get_cmap('tab20', n_epoch)
    return [plt.matplotlib.colors.rgb2hex(cmap(i)) for i in range(n_epoch)]

sess = curr_active_pipeline.sess # global_session

# pos_df = sess.compute_position_laps() # ensures the laps are computed if they need to be:
position_obj = deepcopy(sess.position)
position_obj.compute_higher_order_derivatives()
pos_df = position_obj.compute_smoothed_position_info(N=20) ## Smooth the velocity curve to apply meaningful logic to it
pos_df = position_obj.to_dataframe()
# Drop rows with missing data in columns: 't', 'velocity_x_smooth' and 2 other columns. This occurs from smoothing
pos_df = pos_df.dropna(subset=['t', 'x_smooth', 'velocity_x_smooth', 'acceleration_x_smooth']).reset_index(drop=True)
# curr_laps_df = sess.laps.to_dataframe()

curr_paradigm_df = ensure_dataframe(sess.paradigm)
curr_paradigm_df = curr_paradigm_df[np.logical_not(np.isin(curr_paradigm_df['label'], ['maze_GLOBAL', 'maze']))] ## exclude the global epoch
n_epochs: int = len(curr_paradigm_df)
# epoch_color_strs: List[str] = generate_colors(n_epochs)
epoch_color_strs: List[str] = [ColorFormatConverter.qColor_to_hexstring(v, include_alpha=False) for v in ColormapHelpers.mpl_to_pg_colormap(mpl_cmap_name='tab20', resolution=n_epochs).getColors(mode='qcolor')]
curr_paradigm_df['lap_color'] = "#10FF44"
curr_paradigm_df['lap_color'] = epoch_color_strs
curr_paradigm_df['lap_accent_color'] = '#FFFFFF'
curr_paradigm_df
## Create a new window:
custom_epoch_label_kwargs = dict(epoch_label_position=0.05, epoch_label_rotateAxis=(0,0), epoch_label_anchor=(0.0, 1.0))
epochs_editor = EpochsEditor.init_laps_diagnoser(pos_df, curr_paradigm_df, include_velocity=False, include_accel=False, span=(0.05, 0.95), movable=False, **custom_epoch_label_kwargs)

In [ ]:
from pyphoplacecellanalysis.Pho2D.PyQtPlots.TimeSynchronizedPlotters.Mixins.AnimalTrajectoryPlottingMixin import AnimalTrajectoryPlottingMixin
from pyphoplacecellanalysis.Pho2D.PyQtPlots.TimeSynchronizedPlotters.TimeSynchronizedPositionDecoderPlotter import TimeSynchronizedPositionDecoderPlotter
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.ContainerBased.PhoContainerTool import GenericPyQtGraphContainer
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import build_combined_time_synchronized_Bapun_decoders_window
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.DirectionalPlacefieldGlobalComputationFunctions import DirectionalDecodersContinuouslyDecodedResult
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.SpikeRasterWidgets.Spike2DRaster import SynchronizedPlotMode, Spike2DRaster

# _out_container: GenericPyQtGraphContainer = build_combined_time_synchronized_Bapun_decoders_window(curr_active_pipeline, included_filter_names=['roam', 'sprinkle'], fixed_window_duration = 15.0)
# # _out_container: GenericPyQtGraphContainer = build_combined_time_synchronized_Bapun_decoders_window(curr_active_pipeline, included_filter_names=curr_active_pipeline.active_completed_computation_result_names, fixed_window_duration = 15.0)
# # _out_container: GenericPyQtGraphContainer = build_combined_time_synchronized_Bapun_decoders_window(curr_active_pipeline, included_filter_names=['maze1', 'maze2', 'maze'], fixed_window_duration = 15.0)
# active_2d_plot: Spike2DRaster = _out_container.ui.controlling_widget
# sync_plotters = _out_container.ui.sync_plotters
# # a_sync_plotter: TimeSynchronizedPositionDecoderPlotter = sync_plotters['roam']
# # a_sync_plotter.curr_position



In [ ]:
from pyphoplacecellanalysis.GUI.PyQtPlot.DockingWidgets.DynamicDockDisplayAreaContent import CustomDockDisplayConfig
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import DummyOneStepDecoder


pf2D_Decoder_dict: Dict[str, BasePositionDecoder] = directional_decoders_decode_result.pf1D_Decoder_dict
pseudo3D_decoder: BasePositionDecoder = directional_decoders_decode_result.pseudo2D_decoder
# all_directional_pf1D_Decoder_dict: Dict[str, BasePositionDecoder] = directional_decoders_decode_result.pf1D_Decoder_dict
continuously_decoded_result_cache_dict = directional_decoders_decode_result.continuously_decoded_result_cache_dict
continuously_decoded_pseudo2D_decoder_dict = directional_decoders_decode_result.continuously_decoded_pseudo2D_decoder_dict
## Unpacking a result:
a_time_bin_size: float = list(continuously_decoded_pseudo2D_decoder_dict.keys())[-1] ## ALWAYS GET THE MOST RECENT
all_context_filter_epochs_decoder_result: SingleEpochDecodedResult = continuously_decoded_pseudo2D_decoder_dict[a_time_bin_size] ## ALWAYS GET THE MOST RECENT
marginal_z: NDArray = all_context_filter_epochs_decoder_result.marginal_z.p_x_given_n

# individual_decoder_decoding_results = {k:v for k, v in directional_decoders_decode_result.continuously_decoded_result_cache_dict[1.0].items() if k != 'pseudo2D'}

# one_step_decoder_dummy_dict = {}
# for k, v in individual_decoder_decoding_results.items():
#     one_step_decoder_dummy_dict[k] = DummyOneStepDecoder(xbin=deepcopy(pseudo3D_decoder.xbin), ybin=deepcopy(pseudo3D_decoder.ybin), time_window_centers=deepcopy(all_context_filter_epochs_decoder_result.time_bin_container.centers), p_x_given_n=deepcopy(v.p_x_given_n))



## 2025-10-21 - Plot Laps in 3D

In [59]:
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.ContainerBased.PhoContainerTool import GenericMatplotlibContainer
from pyphoplacecellanalysis.PhoPositionalData.plotting.laps import plot_lap_trajectories_2d, plot_lap_trajectories_3d
from pyphoplacecellanalysis.PhoPositionalData.plotting.laps import _plot_helper_add_arrow
# _out = plot_lap_trajectories_2d(a_sess)



In [60]:
curr_active_pipeline.filtered_sessions

{'roam': DataSession(RatN_Day4_2019-10-15_11-30-06.xml), 'sprinkle': DataSession(RatN_Day4_2019-10-15_11-30-06.xml)}

In [ ]:
# pseudo3D_decoder

k = 'roam'
# k = 'sprinkle'
# k = 'maze_GLOBAL'

a_sess = curr_active_pipeline.filtered_sessions[k]
pos_df = a_sess.position.to_dataframe()
pos_df['speed_xy'] = np.sqrt(np.power(pos_df['velocity_x_smooth'], 2) +  np.power(pos_df['velocity_y_smooth'], 2))

laps_df: pd.DataFrame = ensure_dataframe(a_sess.laps)
# for k, a_sess in curr_active_pipeline.filtered_sessions.items():
#     pos_df = a_sess.position.to_dataframe()
spikes_df = a_sess.spikes_df.spikes.adding_lap_identity_column(laps_epoch_df=a_sess.laps.to_dataframe(), epoch_id_key_name='lap')
# spikes_df

lap_only_pos_df: pd.DataFrame = pos_df.dropna(subset=['lap'], inplace=False)
lap_only_pos_df['lap'] = lap_only_pos_df['lap'].astype(int)
lap_pos_df_dict = lap_only_pos_df.pho.partition_df_dict('lap')
# lap_pos_df_dict
lap_only_pos_df

In [ ]:
laps_df

In [ ]:
lap_dir_2D_dict

In [ ]:
laps_df.pho.partition_df_dict('maze_id')


In [ ]:
# pos_df[['velocity_x_smooth', 'velocity_y_smooth']]

# pos_df['speed_xy'] = np.sqrt(np.power(pos_df['velocity_x_smooth'], 2) +  np.power(pos_df['velocity_y_smooth'], 2))
# pos_df

pos_df.plot(x='t', y='speed_xy')

In [ ]:
# arrow_concentration_kwargs = dict(
#     arrow_skip = 50, time_cmap='viridis',
#     mutation_scale_multiplier = 20, mutation_scale_constant = 1,
# 	arrow_length_multiplier = 0.2, arrow_length_constant = 0.05,
# 	arrow_lw = 0.5,
# )

arrow_concentration_kwargs = dict(
    arrow_skip = 50, time_cmap='viridis',
    mutation_scale_multiplier = 20, mutation_scale_constant = 1,
	arrow_length_multiplier = 0.05, arrow_length_constant = 0.01,
	arrow_lw = 0.5,
)

plot_lap_trajectories_2d_kwargs = dict(
    curr_num_subplots=(6*5), active_page_index=2, fixed_columns = 6,
)

out3: GenericMatplotlibContainer = plot_lap_trajectories_2d(a_sess, **plot_lap_trajectories_2d_kwargs, use_time_gradient_line=True, arrow_concentration_kwargs=arrow_concentration_kwargs)
p3, axs, laps_pages3 = out3.fig, out3.axes, out3.plots_data.laps_pages
p3

In [ ]:
a_sess.laps.to_dataframe()


In [ ]:
# Performed 3 aggregations grouped on column: 'lap'
each_lap_agg_stats_df = lap_only_pos_df.groupby(['lap']).agg(t_count=('t', 'count'), speed_min=('speed', 'min'), speed_max=('speed', 'max')).reset_index()
each_lap_agg_stats_df

In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt
# from matplotlib.collections import LineCollection, PathCollection
# from matplotlib.patches import FancyArrowPatch


# # , time_cmap='viridis', s=50, marker='D', **scatter_kwargs

# # --- Sample Data ---
# t = np.linspace(0, 10, 500)
# x = np.cos(t) + 0.1 * np.random.randn(len(t))
# y = np.sin(t) + 0.1 * np.random.randn(len(t))
# speed = np.gradient(np.sqrt(x**2 + y**2))**2  # fake "speed"

# # --- Create a colored LineCollection based on speed ---
# points = np.array([x, y]).T.reshape(-1, 1, 2)
# segments = np.concatenate([points[:-1], points[1:]], axis=1)
# lc = LineCollection(segments, cmap='viridis', linewidth=2, array=speed)
# fig, ax = plt.subplots(figsize=(7, 7))
# ax.add_collection(lc)

# _out_markers = _helper_add_concentrated_arrows_to_line(ax=ax, t=t, x=x, y=y, speed=speed)


# ax.autoscale()
# ax.set_aspect('equal')
# plt.show()


In [ ]:
viewer, layer, lap_ids = plot_lap_trajectories_3d_napari(a_sess, lap_id_dependent_z_offset=1.0)

In [ ]:
# a_sess.compute_laps_position_df()
# a_sess.compute_spikes_PBEs()

# if 'lap' not in a_sess.spikes_df.columns:
    # spikes_df = a_sess.spikes_df.spikes.adding_lap_identity_column(laps_epoch_df=a_sess.laps.to_dataframe(), epoch_id_key_name='lap')
spikes_df = a_sess.spikes_df.spikes.adding_lap_identity_column(laps_epoch_df=a_sess.laps.to_dataframe(), epoch_id_key_name='lap')
spikes_df


In [ ]:

## single_combined_plot == True mode (mode 1.):
p, laps_pages = plot_lap_trajectories_3d(a_sess, single_combined_plot=True)
p.show()


In [ ]:
from pyvistaqt import BackgroundPlotter, MultiPlotter
# p[0,0]

scenes_output_path = Path('data/3d_scenes').resolve()
assert scenes_output_path.exists()


bg_p: BackgroundPlotter = p[0,0]
# bg_p.export_gltf(scenes_output_path.joinpath('2025-10-21_3d_laps.gltf').as_posix())
bg_p.export_obj(scenes_output_path.joinpath('2025-10-21_3d_laps.obj').as_posix())
bg_p.export_vtkjs(scenes_output_path.joinpath('2025-10-21_3d_laps').as_posix())
# bg_p.export_html(scenes_output_path.joinpath('2025-10-21_3d_laps.html'))


In [ ]:
## single_combined_plot == False mode (mode 2.):        
p2, laps_pages2 = plot_lap_trajectories_3d(a_sess, single_combined_plot=False, curr_num_subplots=len(curr_active_pipeline.sess.laps.lap_id), active_page_index=1)
p2.show()

In [ ]:
plt.close('all')

In [ ]:
# out3.plots.artists['line_artists']

a_linear_index: int = 0


line

In [ ]:
num_subplots: int = len(out3.plots.artists['line_artists'])
for a_linear_index in np.arange(num_subplots):
    _out_markers =  out3.plots.artists['line_markers'][a_linear_index]
    line = out3.plots.artists['line_artists'][a_linear_index]    
    _out_markers.set_sizes(np.atleast_1d(np.full_like(_out_markers.get_sizes(), 0.1)))
    
p3.canvas.draw_idle()

In [ ]:
line

In [ ]:
line = out3.plots.artists['line_artists'][0]
# --- Extract x/y data back ---
segments = line.get_segments()  # list of (N, 2) arrays
xdata = np.concatenate([seg[:, 0] for seg in segments])
ydata = np.concatenate([seg[:, 1] for seg in segments])

xdata, ydata


In [ ]:
line.get_color().shape # (342, 4)
# line.get_colors()

In [ ]:
line_markers = {}
line_markers['start'] = _plot_helper_add_arrow(line, position=0, position_mode='index', direction='right', size=20, color='green') # start
line_markers['end'] = _plot_helper_add_arrow(line, position=None, position_mode='index', direction='right', size=20, color='yellow') # middle
# _plot_helper_add_arrow(line[0], position=curr_lap_num_points, position_mode='index', direction='right', size=20, color='red') # end
line_markers

In [ ]:
p3.canvas.draw_idle()

In [ ]:
from pyphocorehelpers.plotting.media_output_helpers import save_array_as_video

video_out_path = save_array_as_video(array=active_relative_entropy_results['snapshot_occupancy_weighted_tuning_maps'], video_filename='output/videos/snapshot_occupancy_weighted_tuning_maps.avi', isColor=False)
print(f'video_out_path: {video_out_path}')
reveal_in_system_file_manager(video_out_path)


In [ ]:
active_2d_plot.active_embedded_track_pyqtgraph_time_sync_widgets


In [ ]:
print_keys_if_possible('out', _out_container_new.plots, max_depth=2)


In [ ]:
_out_container_new.plots.parent_root_widget
print_keys_if_possible('', _out_container_new, max_depth=3)


In [ ]:
for name, plotter in _out_container_new.ui.sync_plotters.items():
    plotter.plots_data
    # export_pyqtgraph_plot(plotter.ui.root_graphics_layout_widget)

    plotter.export_

In [ ]:
curr_position_decoder_plotter.on_window_changed(5.0, 15.0)

# 2025-10-21 - Spatial/Directional 2D Decoding

In [ ]:
adding_approx_head_dir_columns

In [54]:
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import compute_3d_occupancy_map
# 1. Compute the 3D occupancy map

a_name: str = 'roam'
global_pos_df = deepcopy(curr_active_pipeline.filtered_sessions[a_name].position.to_dataframe())
global_pos_df = global_pos_df.position.adding_approx_head_dir_columns()
global_pf2D = deepcopy(pf2D_Decoder_dict[a_name])
## INPUTS: global_pf2D, global_pos_df

xbin_edges = global_pf2D.xbin
ybin_edges = global_pf2D.ybin
# Create evenly spaced bin edges from 0 to 360
n_dir_bins: int = 8
angle_dir_bin_edges = np.linspace(0, 360, n_dir_bins + 1)

n_xbins: int = len(xbin_edges) - 1
n_ybins: int = len(ybin_edges) - 1
n_dir_bins: int = len(angle_dir_bin_edges) - 1

print(f'n_xbins: {n_xbins}, n_ybins: {n_ybins}, n_dir_bins: {n_dir_bins}')

# Use pd.cut with the explicit bin edges
global_pos_df['head_dir_angle_binned'] = pd.cut(global_pos_df['approx_head_dir_degrees'], bins=angle_dir_bin_edges, labels=False, include_lowest=True)
global_pos_df = global_pos_df.position.adding_binned_position_columns(xbin_edges=xbin_edges, ybin_edges=ybin_edges)
global_pos_df = global_pos_df.dropna(axis='index', subset=['binned_x', 'binned_y', 'head_dir_angle_binned'])
global_pos_df


# occupancy_map, bin_counts = compute_3d_occupancy_map(global_pos_df)

occupancy_map, bin_counts = compute_3d_occupancy_map(global_pos_df, n_x_bins=n_xbins, n_y_bins=n_ybins, n_dir_bins=n_dir_bins)

# Print the shape and counts
print(f"Occupancy map shape: {occupancy_map.shape}")
print(f"Unique bins per dimension: {bin_counts}")



n_xbins: 41, n_ybins: 63, n_dir_bins: 8
Occupancy map shape: (41, 63, 8)
Unique bins per dimension: {'x': 41, 'y': 63, 'dir': 8}


In [56]:
occupancy_map.shape

(41, 63, 8)

In [52]:
global_pos_df.head(25)

,t,x,y,z,lin_pos,dt,velocity_x,acceleration_x,velocity_y,acceleration_y,velocity_z,acceleration_z,x_smooth,y_smooth,z_smooth,velocity_x_smooth,acceleration_x_smooth,velocity_y_smooth,acceleration_y_smooth,velocity_z_smooth,acceleration_z_smooth,speed,binned_x,binned_y,approx_head_dir_degrees,head_dir_angle_binned
890774,7423.119047,103.490906,-48.221642,7.899282,12.258236,0.008333,-2.425258,-57.497112,-5.215258,-81.019633,-1.333079,-101.903424,103.886139,-48.141203,7.863201,-8.300517,-19.402062,0.671798,-41.722047,0.664430,-10.664628,5.751590,39,16,175.372880,3
890775,7423.127380,103.532879,-48.309994,7.868549,12.257953,0.008333,5.036856,895.453419,-10.602261,-646.440234,-3.687957,-282.585221,103.819767,-48.141495,7.866689,-7.964726,40.294833,-0.035019,-84.818063,0.418566,-29.503643,11.737881,39,16,180.251916,4
890776,7423.135714,103.596949,-48.397479,7.827524,12.257671,0.008333,7.688336,318.177554,-10.498198,12.487630,-4.922983,-148.203075,103.766789,-48.151024,7.866053,-6.357259,192.895977,-1.143507,-133.018429,-0.076377,-59.393156,13.012404,39,16,190.196989,4
890777,7423.144047,103.690407,-48.497952,7.789745,12.257388,0.008333,11.214990,423.198283,-12.056731,-187.023949,-4.533438,46.745339,103.726510,-48.170205,7.863225,-4.833525,182.848094,-2.301706,-138.983940,-0.339404,-31.563231,16.466352,39,15,205.463616,4
890778,7423.152380,103.790998,-48.620748,7.775735,12.257106,0.008333,12.070927,102.712435,-14.735536,-321.456460,-1.681222,342.265862,103.696078,-48.200074,7.860695,-3.651830,141.803345,-3.584323,-153.913908,-0.303581,4.298786,19.048446,39,15,224.465496,4
890779,7423.160714,103.928399,-48.771903,7.757646,12.256823,0.008333,16.488028,530.051925,-18.138648,-408.373377,-2.170673,-58.734195,103.682840,-48.242854,7.856473,-1.588598,247.587747,-5.133547,-185.906828,-0.506573,-24.359070,24.512560,39,15,252.805097,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
890793,7423.277380,104.075088,-49.433950,7.984671,12.252868,0.008333,-11.621520,486.323502,8.614696,-374.586440,1.858200,-113.527551,104.435171,-49.495866,7.819063,2.272715,-189.539514,-6.505611,186.801791,1.671485,28.315367,14.466261,39,15,289.256705,6
890794,7423.285714,104.005941,-49.379511,8.009314,12.252586,0.008333,-8.297570,398.873842,6.532680,-249.841737,2.957165,131.875655,104.440340,-49.536373,7.835840,0.620342,-198.284720,-4.860856,197.370567,2.013341,41.022690,10.560567,39,15,277.272771,6
890795,7423.294047,103.970493,-49.344716,8.033260,12.252303,0.008333,-4.253796,485.252807,4.175358,-282.878631,2.873456,-10.045034,104.433308,-49.563570,7.854720,-0.843897,-175.708606,-3.263570,191.674256,2.265540,30.263900,5.960570,39,15,255.501974,5


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 (needed for 3D projection)

qui
    


C:\Users\pho\AppData\Local\Temp\ipykernel_14416\955733580.py:51: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = cm.get_cmap('plasma')


KeyboardInterrupt: 

In [46]:
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import CircularBinnedImageRenderingWindow, plot_spatial_angular_distributions, radial_histogram, draw_radial_lines, plot_directional_occupancy


In [ ]:

# ## INPUTS: occupancy_map, n_xbins, n_ybins n_x_bins=n_xbins, n_y_bins=n_ybins, n_dir_bins=n_dir_bins
# Create sample angular distribution data
# n_x_bins, n_y_bins = 10, 10
# n_angle_bins = 36
# angular_matrix = np.random.rand(n_x_bins, n_y_bins, n_angle_bins)
angular_matrix = deepcopy(occupancy_map)

# Create window
window = CircularBinnedImageRenderingWindow(
    angular_matrix=angular_matrix,
    xbins=np.arange(n_xbins),
    ybins=np.arange(n_ybins),
    n_angle_bins=n_dir_bins,
    name='angular_distribution',
    title='Angular Distribution per Position Bin'
)

window.render_all_circular_heatmaps()


In [49]:
fig, ax = plot_spatial_angular_distributions(occupancy_map, subsample_factor=2)
plt.show()